# Run experiments for FeedForward networks, as a baseline comparison against AttentionModels

In [2]:
%env TMPDIR=/tmp/temp 
#For joblib multi-threading

import os, sys
code_folder=os.path.join(os.getcwd(), "..", "..", "0_code")
sys.path.append(code_folder)

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

import time

env: TMPDIR=/tmp/temp


# Data processing (If re-running code, skip to load data)

In [82]:
# Load Tox21 data
import pandas as pd
import numpy as np
dataset_name="XY_Tox21_CDKPaDEL_processed.csv"
df=pd.read_csv(os.path.join("..", 
                            "..",
                            "rawdata", 
                            dataset_name))

label_columns=range(5,17) #12 Tox21 labels
data_columns=range(17,1504) #1487 CDK+PaDEL descriptors (after filtering)
id_columns=range(5)  

training_ind=np.where(np.isin(df['Type'], ['Training', 'Testing']))[0]
testing_ind=np.where(df['Type']=="Score")[0]
print(f"{dataset_name} has data type 'Training', 'Testing' and 'Score'. Training and Testing (leaderboard) data are reflected in train while Score (Final Evaluation) data is in test.")

X_train=df.iloc[training_ind, data_columns].values
X_test=df.iloc[testing_ind, data_columns].values
y_train=df.iloc[training_ind, label_columns]
y_test=df.iloc[testing_ind, label_columns]
train_id_df=df.iloc[training_ind, id_columns]
test_id_df=df.iloc[testing_ind, id_columns]



XY_Tox21_CDKPaDEL_processed.csv has data type 'Training', 'Testing' and 'Score'. Training and Testing (leaderboard) data are reflected in train while Score (Final Evaluation) data is in test.


In [83]:
#Load tanimoto similarity

dataset_sim_name="XY_Tox21_CDKPaDEL_processed_tanimoto.csv"
df_sim=pd.read_csv(os.path.join("..", 
                                "..",
                                "rawdata", 
                                dataset_sim_name
                               ), 
                   index_col=0
                  )

sim_train_df=df_sim.iloc[training_ind, training_ind+len(id_columns)]
sim_test_df=df_sim.iloc[testing_ind, training_ind+len(id_columns)]

## Impute missing values (as most ML can't deal with missing feature values)
Retain features with only 1 missing value. Drop features with only 1 unique value. Do median imputation. 

In [84]:
def median_imputation(X_train, X_test):
    for col in range(len(X_train[0])):
        med=np.nanmedian(X_train[:,col])
        
        train_nanind=np.where(np.isnan(X_train[:,col]))[0]
        test_nanind=np.where(np.isnan(X_test[:,col]))[0]
        
        if len(train_nanind)>0:
            X_train[train_nanind,col]=med
        if len(test_nanind)>0:
            X_test[test_nanind,col]=med
    return X_train, X_test

def zero_imputation(y):
    for col in range(len(y[0])):
        nan_ind=np.where(np.isnan(y[:,col]))[0]
        if len(nan_ind)>0:
            y[nan_ind,col]=0
    return y
        

In [85]:
retain_feat_ind=np.where(np.sum(np.isnan(X_train),axis=0)<2)[0]

X_train, X_test=median_imputation(X_train[:, retain_feat_ind], X_test[:, retain_feat_ind])



In [86]:
# y_train_impute=zero_imputation(y_train.values)
# y_test_impute=zero_imputation(y_test.values)

## Check for feature correlations

In [87]:
#Remove features that are almost identical to each other (R>0.95)
from scipy.stats import pearsonr
from itertools import combinations
m=X_train.shape[1]
R_mat=np.empty((m,m))
for (i,j) in combinations(range(m), 2):
    r=pearsonr(X_train[:,i], X_train[:,j])
    R_mat[i,j]=r[0]
    R_mat[j,i]=r[0]
for i in range(m):
    R_mat[i,i]=0.    
    
import seaborn as sns
ax= sns.heatmap(R_mat, center=0, cmap='bwr')

In [88]:
R_mat=np.nan_to_num(R_mat, nan=1.)

arr=pd.Series(np.abs(R_mat.flatten()))

print(f"{np.count_nonzero(arr==1)} comparisons with R=1.00")
print(f"{np.count_nonzero(arr>0.95)} comparisons with R>0.95")

136 comparisons with R=1.00
6948 comparisons with R>0.95


In [89]:
# arr=np.where(R_mat==1.)

def split_ind(arr):
    # Given 2D array of pairs of indices, return the lowest index of connected groups of indices
    retain_list=[]
    omit_list=[]
    for i in range(len(arr[0])):
        head, tail=arr[0][i], arr[1][i]
        if head in retain_list:
            omit_list=np.append(omit_list,tail)
        elif tail in retain_list:
            omit_list=np.append(omit_list,head)
        else:
            retain_list=np.append(retain_list, head)
            omit_list=np.append(omit_list,tail)
        retain_list=np.setdiff1d(retain_list, omit_list)
        omit_list=np.unique(omit_list)
    return retain_list, omit_list
    

In [90]:
retain_list2, omit_list2=split_ind(np.where(np.abs(R_mat)>0.99))

In [91]:
len(omit_list2)

345

We omit comparisons with R>0.95

In [92]:
filtered_features=np.setdiff1d(range(m), omit_list2)
X_train_filtered=X_train[:,filtered_features]
X_test_filtered=X_test[:,filtered_features]

print(f"{len(filtered_features)} out of {m} features retained.")

945 out of 1290 features retained.


## Learn Fweights (by fitting COSA model)

In [ ]:
from algorithms import COSA
from importlib import reload
reload(COSA)
cosa_mdl=COSA.NNCosa(Fweight_init="uniform", lam=0.2, n_iter=100,
                                distance_measure="inv_exp_dist",
                     calc_D_ijk=False, threads=-1
                    )
cosa_mdl.fit(X_train_filtered)

Starting on outer iteration 1; beta/eta:0.220
Wchange:0.598, Crit:-401030.862
Wchange:0.142, Crit:-394108.120
Inner loop converged in (or maxed out) at 2 steps
Starting on outer iteration 2; beta/eta:0.240
Wchange:0.061, Crit:-392041.416
Inner loop converged in (or maxed out) at 1 steps
Starting on outer iteration 3; beta/eta:0.260
Wchange:0.032, Crit:-391472.136
Inner loop converged in (or maxed out) at 1 steps
Starting on outer iteration 4; beta/eta:0.280


In [ ]:
cosa_mdl.fit_OOS(X_test_filtered)

In [ ]:
# Fweights=cosa_mdl.output_Fweight()
# cosa_dist=cosa_mdl.output_Dmat()

## Save processed data

In [96]:
save_folder=os.path.join("processed_data"
                        )
new_data_columns=np.array(data_columns)[filtered_features]

processed_df_columns=np.concatenate([id_columns, 
                                     label_columns, 
                                     new_data_columns
                                    ])
processed_df=df
processed_df.iloc[training_ind, new_data_columns]=X_train_filtered
processed_df.iloc[testing_ind, new_data_columns]=X_test_filtered
processed_df=processed_df.iloc[:, processed_df_columns]

processed_df.to_csv(os.path.join(
    save_folder,
    "XY_Tox21_CDKPaDEL_processedFiltered.csv"
))

In [ ]:
# # Save Fweights
import pandas as pd
Fweights=cosa_mdl.output_Fweight()
df= pd.DataFrame(Fweights)
df.to_csv(os.path.join(save_folder,
                       "Tox21_CDKPaDEL_Fweights_train.csv"), 
          index=False)

# import pandas as pd
# Fweights=pd.read_csv("Tox21_Fweights_train.txt")
# cosa_mdl.Fweight=Fweights.values

cosa_dist=cosa_mdl.output_Dmat()
df=pd.DataFrame(cosa_dist)
df.to_csv(os.path.join(save_folder,
                       "Tox21_CDKPaDEL_cosa_dist_train.csv"),
          index=False)

In [ ]:
Fweights_OOS=cosa_mdl.output_Fweight(OOS=True)
df= pd.DataFrame(Fweights_OOS)
df.to_csv(os.path.join(save_folder,
                       "Tox21_CDKPaDEL_Fweights_test.csv"), 
          index=False)

cosa_dist_OOS=cosa_mdl.output_Dmat(OOS=True)
df=pd.DataFrame(cosa_dist_OOS)
df.to_csv(os.path.join(save_folder,
                       "Tox21_CDKPaDEL_cosa_dist_test.csv"),
          index=False)

# Load processed data 
To be updated again when COSA fitting is completed

In [3]:
load_folder=os.path.join("processed_data"
                        )
dataset_name="XY_Tox21_CDKPaDEL_processedFiltered.csv"
load_df=pd.read_csv(os.path.join(load_folder, 
                                 dataset_name
                                ), 
                    index_col=0
                   )

label_columns=range(5,17) #12 Tox21 labels
data_columns=range(17,load_df.shape[1]) #945/1487 filtered CDK+PaDEL descriptors (after filtering)
id_columns=range(5)  

training_ind=np.where(np.isin(load_df['Type'], ['Training', 'Testing']))[0]
testing_ind=np.where(load_df['Type']=="Score")[0]
print(f"{dataset_name} has data type 'Training', 'Testing' and 'Score'. Training and Testing (leaderboard) data are reflected in train while Score (Final Evaluation) data is in test.")

X_train=load_df.iloc[training_ind, data_columns].values
X_test=load_df.iloc[testing_ind, data_columns].values
y_train=load_df.iloc[training_ind, label_columns]
y_test=load_df.iloc[testing_ind, label_columns]
train_id_df=load_df.iloc[training_ind, id_columns]
test_id_df=load_df.iloc[testing_ind, id_columns]


Fweights_df=pd.read_csv(os.path.join(load_folder, 
                                     "Tox21_CDKPaDEL_Fweights_train.csv"
                                    ))
Fweights_train=Fweights_df[load_df.columns[data_columns]].values

Fweights_OOS_df=pd.read_csv(os.path.join(load_folder,
                                          "Tox21_CDKPaDEL_Fweights_test.csv"
                                         ))
Fweights_test=Fweights_OOS_df[load_df.columns[data_columns]].values

XY_Tox21_CDKPaDEL_processedFiltered.csv has data type 'Training', 'Testing' and 'Score'. Training and Testing (leaderboard) data are reflected in train while Score (Final Evaluation) data is in test.


# Train deep feed forward networks

## Model set-up

In [6]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [10]:
#Updated 6th Jan 2021 (Edited Line 71 to Line 72. Reduce_mean instead of mean, to preserve the required rank)

from tensorflow.keras import backend as K
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
epsilon=K.epsilon

def get_weights_dicts(Y):
    weights_dicts=[]
    for j in range(Y.shape[1]):
        weight_zero, weight_one = _get_label_weights(Y[:,j])
        d={'weight_zero':weight_zero,
           'weight_one':weight_one
          }
        weights_dicts.append(d)
    return weights_dicts
def _get_label_weights(y):
    #Get label weights for majority and minority class using the following:
        #major_weight=n/(n_major*2)
        #minor_weight=n*(n_major/n_minor)/(n_major*2)
        #NaN weights are set to zero
    y1=y[~np.isnan(y)]
    n=len(y1)
    n_zero=np.count_nonzero(np.isclose(y1,0))
    n_one=np.count_nonzero(np.isclose(y1,1))
    if n_zero>n_one:
        weight_zero=n/(n_zero*2)
        weight_one=n*(n_zero/n_one)/(n_zero*2)
    else:
        weight_zero=n*(n_one/n_zero)/(n_one*2)
        weight_one=n/(n_one*2)
    return weight_zero, weight_one    

class BinaryCrossEntropyIgnoreNaN(tf.keras.losses.Loss):
    def __init__(self, weights_dicts=None, axis=0, **kwargs):
        super(BinaryCrossEntropyIgnoreNaN, self).__init__(**kwargs)
        self.weights_dicts=weights_dicts
        self.axis=axis        

    def __call__(self, target, output, sample_weight=None):
        #Binary cross entropy that ignores Nan and replaces with mini-batch Nan with 0
        #modified from tf.python.keras.backend.binary_crossentropy
        
        ##NEED TO TEST THIS CODE MORE THOROUGHLY
        target=tf.convert_to_tensor(target)
        output=tf.convert_to_tensor(output)
        if len(target.shape)==1:
            target=tf.expand_dims(target, 1)
            output=tf.expand_dims(output, 1)
        epsilon_ = tf.constant(epsilon(), dtype=output.dtype.base_dtype)
        output=tf.clip_by_value(output, epsilon_, 1. - epsilon_)

        #Compute cross entropy from probabilities
        bce=target * tf.math.log(output+epsilon_)
        bce+=(1-target)* tf.math.log(1-output+epsilon_)

        bce=tf.where(tf.math.is_nan(-bce), epsilon(), -bce)
        if self.weights_dicts is not None:
            sample_weight=tf.cast(tf.where(target==0.,1.,0.)*[self.weights_dicts[i]['weight_zero'] for i in range(len(self.weights_dicts))], dtype=target.dtype)
            sample_weight+=tf.cast(tf.where(target==1., 1., 0.)*[self.weights_dicts[i]['weight_one'] for i in range(len(self.weights_dicts))], dtype=target.dtype)
            bce=tf.multiply(sample_weight, bce)
#         return tf.keras.backend.mean(bce, axis=self.axis)  
        return tf.math.reduce_mean(bce)

    def call(self, target, output, sample_weight=None):
        return self(target, output, sample_weight=None)

## Load single label

In [11]:
label='NR-AR'

train_inds=~np.isnan(y_train[label])
test_inds=~np.isnan(y_test[label])

train_data=X_train[train_inds,:]
train_targets=y_train[label][train_inds].astype(np.float32)
test_data=X_test[test_inds,:]
test_targets=y_test[label][test_inds].astype(np.float32)
#train_Fweights
#test_Fweights

weights_dicts=get_weights_dicts(np.expand_dims(train_targets,1))


In [ ]:
#Temporarily use this file first. 
#There are fewer chemicals in this file

In [ ]:
load_folder=os.path.join("ARE_v2"
                        )

#Load Tox21 processed data (ARE)
processed_data_df=pd.read_csv(os.path.join(load_folder, 
                                           "Tox21_CDKPaDEL_processedData_AREv2.csv"
                                          ), index_col=0
                             )
label_ind=processed_data_df.shape[1]-2 

feat_ind=range(processed_data_df.shape[1]-2)

X_train_ARE2=(processed_data_df.query("Type=='Training'")
             .iloc[:,feat_ind]).values
X_test_ARE2=(processed_data_df.query("Type=='Testing'")
            .iloc[:,feat_ind]).values
y_train_ARE2=(processed_data_df.query("Type=='Training'")
             .iloc[:,label_ind]).values
y_test_ARE2=(processed_data_df.query("Type=='Testing'")
            .iloc[:,label_ind]).values

#Load COSA feature weights and distances for Tox21 processed data (ARE)
Fweights_train_df=pd.read_csv(os.path.join(load_folder,
                                     "Tox21_CDKPaDEL_Fweights_ARE_train2.csv",                                 
                                    )
                       )
Fweights_train=Fweights_train_df.iloc[:,:-1].values


Fweights_test_df=pd.read_csv(os.path.join(load_folder, 
                                          "Tox21_CDKPaDEL_FweightsOOS_ARE_test2.csv"
                                         ))
Fweights_test=Fweights_test_df.iloc[:,:-1]

# temp_ind=Fweights_df['ind']




## Model architecture

In [12]:
# n_feat = train_data.shape[1]
# n_attention = 20 #Reduced from 20 to 10. 10 works better
# n_attention_hidden=20
# n_attention_out=1
# n_concat_hidden=2048
# n_hidden1 =512
# n_hidden2 = 64 #Added 2nd hidden layer
# momentum=0.8
# learning_rate=0.001

# n_batch=32

n_feat = train_tensor.shape[1]
n_attention = 20 #Reduced from 20 to 10. 10 works better
n_attention_hidden=20
n_attention_out=1
n_concat_hidden=4096
n_hidden1 =1024
n_hidden2 = 128 #Added 2nd hidden layer
momentum=0.8
learning_rate=0.001

n_batch=32

save_folder=os.path.join(time.strftime("%y%m%d_TrainingDeep",
                                       time.localtime()))
checkpoint_path = os.path.join(save_folder, 
                               "Dense_{}".format(label),
                               )

try: 
    os.mkdir(save_folder) 
except OSError as error: 
    print(error) 
    
try:
    os.mkdir(checkpoint_path)
except OSError as error:
    print(error)

[WinError 183] Cannot create a file when that file already exists: '210120_TrainingDeep'
[WinError 183] Cannot create a file when that file already exists: '210120_TrainingDeep\\Dense_NR-AR'


In [147]:
from tensorflow.keras.layers import LeakyReLU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.regularizers import l1

np.random.seed(0)

dense_model=Sequential()
dense_model.add(Dense(n_attention*n_attention_hidden, 
                      activation="gelu",
                      kernel_initializer=VarianceScaling(),
                      kernel_regularizer=l1(1E-5),
                      bias_regularizer=l1(1E-5),
#                  input_shape=(n_feat,),
                ))
dense_model.add(Dropout(0.1))
dense_model.add(Dense(n_attention*n_feat, activation="gelu",
                      kernel_initializer=VarianceScaling(),
                      kernel_regularizer=l1(1E-5),
                      bias_regularizer=l1(1E-5),
                     ))
# dense_model.add(LeakyReLU())
dense_model.add(Dense(n_concat_hidden, activation="gelu",  
                      kernel_initializer=VarianceScaling(),
                      kernel_regularizer=l1(1E-5),
                      bias_regularizer=l1(1E-5),   
#                       activity_regularizer=l1(1E-5)                      
                     ))
# dense_model.add(LeakyReLU())
dense_model.add(Dropout(0.1))
dense_model.add(Dense(n_hidden1, activation="gelu",
                      kernel_initializer=VarianceScaling(),
                      kernel_regularizer=l1(1E-5),
                      bias_regularizer=l1(1E-5),
                     ))
# dense_model.add(LeakyReLU())
dense_model.add(Dropout(0.1))
dense_model.add(Dense(n_hidden2, activation="gelu",
                      kernel_initializer=VarianceScaling(),
                      kernel_regularizer=l1(1E-5),
                      bias_regularizer=l1(1E-5),
                     ))
# dense_model.add(LeakyReLU())
dense_model.add(Dropout(0.1))
dense_model.add(Dense(1, activation="sigmoid",
                      kernel_initializer=VarianceScaling(),
                      kernel_regularizer=l1(1E-5),
                      bias_regularizer=l1(1E-5)
                     ))

loss_fn=BinaryCrossEntropyIgnoreNaN(weights_dicts=weights_dicts)

dense_model.compile(tf.optimizers.Adam(learning_rate=learning_rate,),
                   loss=loss_fn, 
                    metrics=['accuracy', 'AUC'],
              )


In [19]:
from tensorflow.keras import callbacks

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_auc',
                                                 mode='max',
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)

csv_filename = os.path.join(checkpoint_path,
                            "training_log.csv"
                            )
csvlogger_callback = tf.keras.callbacks.CSVLogger(filename=csv_filename, append=True)


n_epoch=100

dense_model.fit(train_data,
                train_targets,
                epochs=n_epoch, 
                batch_size=n_batch, 
                validation_data=(test_data, test_targets),
                verbose=2,
                shuffle=True,
                callbacks=[csvlogger_callback,
                           cp_callback
                          ])

Epoch 1/100
292/292 - 8s - loss: 3.6190 - accuracy: 0.7750 - auc: 0.7640 - val_loss: 1.7328 - val_accuracy: 0.9009 - val_auc: 0.6670

Epoch 00001: val_auc improved from -inf to 0.66704, saving model to 210120_TrainingDeep\Dense_NR-AR
Epoch 2/100
292/292 - 6s - loss: 1.2129 - accuracy: 0.9214 - auc: 0.7988 - val_loss: 0.8358 - val_accuracy: 0.9635 - val_auc: 0.6767

Epoch 00002: val_auc improved from 0.66704 to 0.67673, saving model to 210120_TrainingDeep\Dense_NR-AR
Epoch 3/100
292/292 - 6s - loss: 0.8040 - accuracy: 0.8857 - auc: 0.8220 - val_loss: 0.6600 - val_accuracy: 0.9200 - val_auc: 0.6929

Epoch 00003: val_auc improved from 0.67673 to 0.69287, saving model to 210120_TrainingDeep\Dense_NR-AR
Epoch 4/100
292/292 - 7s - loss: 0.6567 - accuracy: 0.8992 - auc: 0.8352 - val_loss: 0.5816 - val_accuracy: 0.9496 - val_auc: 0.7131

Epoch 00004: val_auc improved from 0.69287 to 0.71307, saving model to 210120_TrainingDeep\Dense_NR-AR
Epoch 5/100
292/292 - 7s - loss: 0.6917 - accuracy: 0.8

Epoch 43/100
292/292 - 6s - loss: 0.5706 - accuracy: 0.9078 - auc: 0.8721 - val_loss: 0.5181 - val_accuracy: 0.9374 - val_auc: 0.7285

Epoch 00043: val_auc did not improve from 0.79204
Epoch 44/100
292/292 - 6s - loss: 0.5129 - accuracy: 0.9098 - auc: 0.8751 - val_loss: 0.6038 - val_accuracy: 0.8365 - val_auc: 0.7448

Epoch 00044: val_auc did not improve from 0.79204
Epoch 45/100
292/292 - 6s - loss: 0.5392 - accuracy: 0.9018 - auc: 0.8772 - val_loss: 0.5358 - val_accuracy: 0.9113 - val_auc: 0.7388

Epoch 00045: val_auc did not improve from 0.79204
Epoch 46/100
292/292 - 7s - loss: 0.6074 - accuracy: 0.8923 - auc: 0.8663 - val_loss: 1.3972 - val_accuracy: 0.8817 - val_auc: 0.7557

Epoch 00046: val_auc did not improve from 0.79204
Epoch 47/100
292/292 - 7s - loss: 0.9042 - accuracy: 0.8627 - auc: 0.8415 - val_loss: 0.6388 - val_accuracy: 0.9687 - val_auc: 0.6503

Epoch 00047: val_auc did not improve from 0.79204
Epoch 48/100
292/292 - 7s - loss: 0.6723 - accuracy: 0.9099 - auc: 0.8713 -

292/292 - 6s - loss: 0.5357 - accuracy: 0.8950 - auc: 0.8990 - val_loss: 0.5362 - val_accuracy: 0.8870 - val_auc: 0.7420

Epoch 00087: val_auc did not improve from 0.84488
Epoch 88/100
292/292 - 6s - loss: 0.5020 - accuracy: 0.9074 - auc: 0.9066 - val_loss: 0.6150 - val_accuracy: 0.9496 - val_auc: 0.7490

Epoch 00088: val_auc did not improve from 0.84488
Epoch 89/100
292/292 - 6s - loss: 0.5532 - accuracy: 0.9305 - auc: 0.8829 - val_loss: 0.5763 - val_accuracy: 0.9374 - val_auc: 0.7509

Epoch 00089: val_auc did not improve from 0.84488
Epoch 90/100
292/292 - 6s - loss: 0.5193 - accuracy: 0.9069 - auc: 0.9118 - val_loss: 0.5677 - val_accuracy: 0.9391 - val_auc: 0.7297

Epoch 00090: val_auc did not improve from 0.84488
Epoch 91/100
292/292 - 7s - loss: 0.5012 - accuracy: 0.9140 - auc: 0.9178 - val_loss: 0.5172 - val_accuracy: 0.9548 - val_auc: 0.7080

Epoch 00091: val_auc did not improve from 0.84488
Epoch 92/100
292/292 - 7s - loss: 0.7062 - accuracy: 0.9060 - auc: 0.9017 - val_loss: 0.

### Loop training for all single-labels

In [66]:
def initialize_model(weights_dicts,
                     n_attention=n_attention, 
                     n_attention_hidden=n_attention_hidden, 
                     n_feat=n_feat,
                     n_concat_hidden=n_concat_hidden,
                     n_hidden1=n_hidden1, 
                     n_hidden2=n_hidden2,
                    ):
    dense_model=Sequential()
    dense_model.add(Dense(n_attention*n_attention_hidden, 
                          activation="gelu",
                          kernel_initializer=VarianceScaling(),
                          kernel_regularizer=l1(1E-5),
                          bias_regularizer=l1(1E-5),
    #                  input_shape=(n_feat,),
                    ))
    dense_model.add(Dense(n_attention*n_feat*n_attention_out, activation="gelu",
                          kernel_initializer=VarianceScaling(),
                          kernel_regularizer=l1(1E-5),
                          bias_regularizer=l1(1E-5),
                         ))
    # dense_model.add(LeakyReLU())
    dense_model.add(Dense(n_concat_hidden, activation="gelu",  
                          kernel_initializer=VarianceScaling(),
                          kernel_regularizer=l1(1E-5),
                          bias_regularizer=l1(1E-5),   
    #                       activity_regularizer=l1(1E-5)                      
                         ))
    # dense_model.add(LeakyReLU())
    dense_model.add(Dropout(0.1))
    dense_model.add(Dense(n_hidden1, activation="gelu",
                          kernel_initializer=VarianceScaling(),
                          kernel_regularizer=l1(1E-5),
                          bias_regularizer=l1(1E-5),
                         ))
    # dense_model.add(LeakyReLU())
    dense_model.add(Dropout(0.1))
    dense_model.add(Dense(n_hidden2, activation="gelu",
                          kernel_initializer=VarianceScaling(),
                          kernel_regularizer=l1(1E-5),
                          bias_regularizer=l1(1E-5),
                         ))
    # dense_model.add(LeakyReLU())
    dense_model.add(Dropout(0.1))
    dense_model.add(Dense(1, activation="sigmoid",
                          kernel_initializer=VarianceScaling(),
                          kernel_regularizer=l1(1E-5),
                          bias_regularizer=l1(1E-5)
                         ))

    loss_fn=BinaryCrossEntropyIgnoreNaN(weights_dicts=weights_dicts)

    dense_model.compile(tf.optimizers.Adam(learning_rate=learning_rate,),
                       loss=loss_fn, 
                        metrics=['accuracy', 'AUC'],
                  )
    return dense_model

In [68]:
# Repeat above for all labels
n_epoch = 100
tmp_file = os.path.join(save_folder,
"{}_Dense_tmpfile.csv".format(time.strftime("%y%m%d", time.localtime()),
                     ))

with open(tmp_file, 'w') as tmpfile:
    tmpfile.write(",".join(["Label", "Best_AUC", "/n"]))

for label in y_train.columns:
    # Load data
    train_inds = ~np.isnan(y_train[label])
    test_inds = ~np.isnan(y_test[label])

    train_data = X_train[train_inds, :]
    train_targets = y_train[label][train_inds].astype(np.float32)
    test_data = X_test[test_inds, :]
    test_targets = y_test[label][test_inds].astype(np.float32)
    # train_Fweights
    # test_Fweights

    weights_dicts = get_weights_dicts(np.expand_dims(train_targets, 1))

    # Set save folders
    checkpoint_path = os.path.join(save_folder, "{}_Dense_{}".format(time.strftime("%y%m%d", time.localtime()),
                     label)
                                  )
    try:
        os.mkdir(checkpoint_path)
    except OSError as error:
        print(error)

    np.random.seed(0)

    dense_model_=initialize_model(weights_dicts = weights_dicts)


    cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 monitor = 'val_auc',
                                                 mode = 'max',
                                                 save_best_only = True,
                                                 save_weights_only = True,
                                                 verbose = 1)

    csv_filename=os.path.join(checkpoint_path, "training_log.csv")
    csvlogger_callback=tf.keras.callbacks.CSVLogger(
        filename = csv_filename, append = True)

    dense_model_.fit(train_data,
                    train_targets,
                    epochs=n_epoch, 
                    batch_size=n_batch, 
                    validation_data=(test_data, test_targets),
                    verbose=2,
                    shuffle=True,
                    callbacks=[csvlogger_callback,
                               cp_callback
                              ])
    
    with open(tmp_file, 'a') as tmpfile:
        tmpfile.write(",".join([label, str(cp_callback.best)+"/n"]))
    
    del dense_model_

[WinError 183] Cannot create a file when that file already exists: '210120_TrainingDeep\\210120_Dense_NR-AR'
Epoch 1/100
292/292 - 9s - loss: 4.3231 - accuracy: 0.7380 - auc: 0.7148 - val_loss: 2.4061 - val_accuracy: 0.9565 - val_auc: 0.6796

Epoch 00001: val_auc improved from -inf to 0.67962, saving model to 210120_TrainingDeep\210120_Dense_NR-AR
Epoch 2/100
292/292 - 7s - loss: 1.7785 - accuracy: 0.8925 - auc: 0.8014 - val_loss: 1.2416 - val_accuracy: 0.9722 - val_auc: 0.7288

Epoch 00002: val_auc improved from 0.67962 to 0.72876, saving model to 210120_TrainingDeep\210120_Dense_NR-AR
Epoch 3/100
292/292 - 7s - loss: 1.0809 - accuracy: 0.9184 - auc: 0.8234 - val_loss: 0.8635 - val_accuracy: 0.9722 - val_auc: 0.7126

Epoch 00003: val_auc did not improve from 0.72876
Epoch 4/100
292/292 - 7s - loss: 0.8160 - accuracy: 0.9388 - auc: 0.8261 - val_loss: 0.7276 - val_accuracy: 0.9722 - val_auc: 0.7349

Epoch 00004: val_auc improved from 0.72876 to 0.73490, saving model to 210120_TrainingDe


Epoch 00042: val_auc did not improve from 0.77442
Epoch 43/100
292/292 - 7s - loss: 0.4521 - accuracy: 0.8786 - auc: 0.9120 - val_loss: 0.5806 - val_accuracy: 0.6957 - val_auc: 0.6780

Epoch 00043: val_auc did not improve from 0.77442
Epoch 44/100
292/292 - 7s - loss: 0.4348 - accuracy: 0.8857 - auc: 0.9145 - val_loss: 0.5047 - val_accuracy: 0.8017 - val_auc: 0.7602

Epoch 00044: val_auc did not improve from 0.77442
Epoch 45/100
292/292 - 7s - loss: 0.4260 - accuracy: 0.8721 - auc: 0.9226 - val_loss: 0.5671 - val_accuracy: 0.9026 - val_auc: 0.6089

Epoch 00045: val_auc did not improve from 0.77442
Epoch 46/100
292/292 - 7s - loss: 0.4301 - accuracy: 0.8806 - auc: 0.9190 - val_loss: 0.5174 - val_accuracy: 0.8609 - val_auc: 0.6955

Epoch 00046: val_auc did not improve from 0.77442
Epoch 47/100
292/292 - 7s - loss: 0.4080 - accuracy: 0.8740 - auc: 0.9321 - val_loss: 0.6631 - val_accuracy: 0.6748 - val_auc: 0.6881

Epoch 00047: val_auc did not improve from 0.77442
Epoch 48/100
292/292 - 7


Epoch 00086: val_auc did not improve from 0.79744
Epoch 87/100
292/292 - 7s - loss: 0.8122 - accuracy: 0.8896 - auc: 0.7853 - val_loss: 0.7152 - val_accuracy: 0.8991 - val_auc: 0.7308

Epoch 00087: val_auc did not improve from 0.79744
Epoch 88/100
292/292 - 7s - loss: 0.9021 - accuracy: 0.9099 - auc: 0.8053 - val_loss: 0.9310 - val_accuracy: 0.9287 - val_auc: 0.7395

Epoch 00088: val_auc did not improve from 0.79744
Epoch 89/100
292/292 - 7s - loss: 0.8585 - accuracy: 0.9093 - auc: 0.8303 - val_loss: 0.6901 - val_accuracy: 0.9235 - val_auc: 0.6002

Epoch 00089: val_auc did not improve from 0.79744
Epoch 90/100
292/292 - 7s - loss: 0.8519 - accuracy: 0.9454 - auc: 0.4984 - val_loss: 0.6183 - val_accuracy: 0.9791 - val_auc: 0.5000

Epoch 00090: val_auc did not improve from 0.79744
Epoch 91/100
292/292 - 6s - loss: 0.6907 - accuracy: 0.9267 - auc: 0.8006 - val_loss: 0.5488 - val_accuracy: 0.9583 - val_auc: 0.7408

Epoch 00091: val_auc did not improve from 0.79744
Epoch 92/100
292/292 - 7


Epoch 00027: val_auc did not improve from 0.90153
Epoch 28/100
257/257 - 6s - loss: 0.4145 - accuracy: 0.8493 - auc: 0.9306 - val_loss: 0.5497 - val_accuracy: 0.8138 - val_auc: 0.8921

Epoch 00028: val_auc did not improve from 0.90153
Epoch 29/100
257/257 - 6s - loss: 0.4324 - accuracy: 0.8328 - auc: 0.9259 - val_loss: 0.6034 - val_accuracy: 0.8641 - val_auc: 0.8904

Epoch 00029: val_auc did not improve from 0.90153
Epoch 30/100
257/257 - 6s - loss: 0.4255 - accuracy: 0.8421 - auc: 0.9281 - val_loss: 0.5127 - val_accuracy: 0.7903 - val_auc: 0.8847

Epoch 00030: val_auc did not improve from 0.90153
Epoch 31/100
257/257 - 6s - loss: 0.4292 - accuracy: 0.8474 - auc: 0.9304 - val_loss: 0.5040 - val_accuracy: 0.8188 - val_auc: 0.8964

Epoch 00031: val_auc did not improve from 0.90153
Epoch 32/100
257/257 - 6s - loss: 0.4382 - accuracy: 0.8389 - auc: 0.9235 - val_loss: 0.5436 - val_accuracy: 0.8389 - val_auc: 0.8880

Epoch 00032: val_auc did not improve from 0.90153
Epoch 33/100
257/257 - 6

257/257 - 6s - loss: 0.3294 - accuracy: 0.8765 - auc: 0.9581 - val_loss: 0.6565 - val_accuracy: 0.7483 - val_auc: 0.8643

Epoch 00072: val_auc did not improve from 0.90153
Epoch 73/100
257/257 - 6s - loss: 0.6633 - accuracy: 0.7836 - auc: 0.9188 - val_loss: 0.9384 - val_accuracy: 0.7886 - val_auc: 0.8822

Epoch 00073: val_auc did not improve from 0.90153
Epoch 74/100
257/257 - 6s - loss: 0.4929 - accuracy: 0.8556 - auc: 0.9503 - val_loss: 0.6608 - val_accuracy: 0.7852 - val_auc: 0.8737

Epoch 00074: val_auc did not improve from 0.90153
Epoch 75/100
257/257 - 6s - loss: 0.3768 - accuracy: 0.8727 - auc: 0.9599 - val_loss: 0.6288 - val_accuracy: 0.7785 - val_auc: 0.9030

Epoch 00075: val_auc improved from 0.90153 to 0.90305, saving model to 210120_TrainingDeep\210120_Dense_NR-AhR
Epoch 76/100
257/257 - 6s - loss: 0.3498 - accuracy: 0.8740 - auc: 0.9609 - val_loss: 0.7538 - val_accuracy: 0.7752 - val_auc: 0.8811

Epoch 00076: val_auc did not improve from 0.90305
Epoch 77/100
257/257 - 6s -

...

257/257 - 6s - loss: 0.3566 - accuracy: 0.8672 - auc: 0.9567 - val_loss: 1.0749 - val_accuracy: 0.8389 - val_auc: 0.7191

Epoch 00095: val_auc did not improve from 0.81191
Epoch 96/100
257/257 - 6s - loss: 0.3601 - accuracy: 0.8691 - auc: 0.9536 - val_loss: 0.8648 - val_accuracy: 0.7114 - val_auc: 0.7402

Epoch 00096: val_auc did not improve from 0.81191
Epoch 97/100
257/257 - 6s - loss: 0.3445 - accuracy: 0.8792 - auc: 0.9595 - val_loss: 0.9937 - val_accuracy: 0.7685 - val_auc: 0.7178

Epoch 00097: val_auc did not improve from 0.81191
Epoch 98/100
257/257 - 6s - loss: 0.3597 - accuracy: 0.8595 - auc: 0.9534 - val_loss: 0.8146 - val_accuracy: 0.7919 - val_auc: 0.7362

Epoch 00098: val_auc did not improve from 0.81191
Epoch 99/100
257/257 - 6s - loss: 0.3713 - accuracy: 0.8654 - auc: 0.9511 - val_loss: 0.9996 - val_accuracy: 0.6846 - val_auc: 0.6708

Epoch 00099: val_auc did not improve from 0.81191
Epoch 100/100
257/257 - 6s - loss: 0.3693 - accuracy: 0.8677 - auc: 0.9538 - val_loss: 1


Epoch 00037: val_auc did not improve from 0.93021
Epoch 38/100
230/230 - 5s - loss: 0.3528 - accuracy: 0.8946 - auc: 0.9590 - val_loss: 0.4293 - val_accuracy: 0.8701 - val_auc: 0.9224

Epoch 00038: val_auc did not improve from 0.93021
Epoch 39/100
230/230 - 5s - loss: 0.3494 - accuracy: 0.8986 - auc: 0.9604 - val_loss: 0.4223 - val_accuracy: 0.8814 - val_auc: 0.9292

Epoch 00039: val_auc did not improve from 0.93021
Epoch 40/100
230/230 - 5s - loss: 0.3389 - accuracy: 0.8975 - auc: 0.9610 - val_loss: 0.4342 - val_accuracy: 0.8719 - val_auc: 0.9232

Epoch 00040: val_auc did not improve from 0.93021
Epoch 41/100
230/230 - 5s - loss: 0.3468 - accuracy: 0.8946 - auc: 0.9594 - val_loss: 0.5784 - val_accuracy: 0.6911 - val_auc: 0.9207

Epoch 00041: val_auc did not improve from 0.93021
Epoch 42/100
230/230 - 5s - loss: 0.3521 - accuracy: 0.8975 - auc: 0.9598 - val_loss: 0.5073 - val_accuracy: 0.8004 - val_auc: 0.9251

Epoch 00042: val_auc did not improve from 0.93021
Epoch 43/100
230/230 - 5

230/230 - 5s - loss: 0.2662 - accuracy: 0.9233 - auc: 0.9757 - val_loss: 0.5510 - val_accuracy: 0.7721 - val_auc: 0.9065

Epoch 00082: val_auc did not improve from 0.93021
Epoch 83/100
230/230 - 5s - loss: 0.2765 - accuracy: 0.9141 - auc: 0.9730 - val_loss: 0.7198 - val_accuracy: 0.8625 - val_auc: 0.8796

Epoch 00083: val_auc did not improve from 0.93021
Epoch 84/100
230/230 - 5s - loss: 0.2827 - accuracy: 0.9138 - auc: 0.9734 - val_loss: 0.4703 - val_accuracy: 0.8399 - val_auc: 0.9000

Epoch 00084: val_auc did not improve from 0.93021
Epoch 85/100
230/230 - 5s - loss: 0.2599 - accuracy: 0.9250 - auc: 0.9762 - val_loss: 0.5350 - val_accuracy: 0.8192 - val_auc: 0.8962

Epoch 00085: val_auc did not improve from 0.93021
Epoch 86/100
230/230 - 5s - loss: 0.2638 - accuracy: 0.9216 - auc: 0.9756 - val_loss: 0.4603 - val_accuracy: 0.7966 - val_auc: 0.9120

Epoch 00086: val_auc did not improve from 0.93021
Epoch 87/100
230/230 - 5s - loss: 0.2652 - accuracy: 0.9190 - auc: 0.9761 - val_loss: 0.


Epoch 00024: val_auc did not improve from 0.83131
Epoch 25/100
271/271 - 6s - loss: 0.7780 - accuracy: 0.9385 - auc: 0.4917 - val_loss: 0.7909 - val_accuracy: 0.9337 - val_auc: 0.5000

Epoch 00025: val_auc did not improve from 0.83131
Epoch 26/100
271/271 - 6s - loss: 0.7609 - accuracy: 0.8272 - auc: 0.4897 - val_loss: 0.7831 - val_accuracy: 0.9337 - val_auc: 0.5000

Epoch 00026: val_auc did not improve from 0.83131
Epoch 27/100
271/271 - 6s - loss: 0.7559 - accuracy: 0.9385 - auc: 0.4873 - val_loss: 0.7798 - val_accuracy: 0.9337 - val_auc: 0.5000

Epoch 00027: val_auc did not improve from 0.83131
Epoch 28/100
271/271 - 6s - loss: 0.7534 - accuracy: 0.6798 - auc: 0.4946 - val_loss: 0.7779 - val_accuracy: 0.9337 - val_auc: 0.5000

Epoch 00028: val_auc did not improve from 0.83131
Epoch 29/100
271/271 - 6s - loss: 0.7521 - accuracy: 0.8570 - auc: 0.4826 - val_loss: 0.7761 - val_accuracy: 0.9337 - val_auc: 0.5000

Epoch 00029: val_auc did not improve from 0.83131
Epoch 30/100
271/271 - 6


Epoch 00068: val_auc improved from 0.83253 to 0.83821, saving model to 210120_TrainingDeep\210120_Dense_SR-p53
Epoch 69/100
271/271 - 6s - loss: 0.7532 - accuracy: 0.7036 - auc: 0.8159 - val_loss: 0.7281 - val_accuracy: 0.4080 - val_auc: 0.8397

Epoch 00069: val_auc improved from 0.83821 to 0.83965, saving model to 210120_TrainingDeep\210120_Dense_SR-p53
Epoch 70/100
271/271 - 6s - loss: 0.9168 - accuracy: 0.6822 - auc: 0.7088 - val_loss: 0.8664 - val_accuracy: 0.7944 - val_auc: 0.7349

Epoch 00070: val_auc did not improve from 0.83965
Epoch 71/100
271/271 - 6s - loss: 0.8909 - accuracy: 0.6318 - auc: 0.7809 - val_loss: 0.8495 - val_accuracy: 0.5406 - val_auc: 0.8369

Epoch 00071: val_auc did not improve from 0.83965
Epoch 72/100
271/271 - 6s - loss: 0.7544 - accuracy: 0.6583 - auc: 0.8050 - val_loss: 0.7465 - val_accuracy: 0.8093 - val_auc: 0.8306

Epoch 00072: val_auc did not improve from 0.83965
Epoch 73/100
271/271 - 6s - loss: 0.7741 - accuracy: 0.6718 - auc: 0.7851 - val_loss: 0

In [81]:
pd.read_csv(tmp_file)


,Label,Best_AUC
0,NR-AR,0.797439
1,NR-AhR,0.903049
2,NR-AR-LBD,0.856840
3,NR-ER,0.838359
4,NR-ER-LBD,0.752601
5,NR-Aromatase,0.831757
6,NR-PPAR-gamma,0.753404
7,SR-ARE,0.784257
8,SR-ATAD5,0.815712
9,SR-HSE,0.811913


# Train Locality-Sensitive Deep Learner

## Model set-up

In [17]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [18]:
#Updated 6th Jan 2021 (Edited Line 71 to Line 72. Reduce_mean instead of mean, to preserve the required rank)

from tensorflow.keras import backend as K
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
epsilon=K.epsilon

def get_weights_dicts(Y):
    weights_dicts=[]
    for j in range(Y.shape[1]):
        weight_zero, weight_one = _get_label_weights(Y[:,j])
        d={'weight_zero':weight_zero,
           'weight_one':weight_one
          }
        weights_dicts.append(d)
    return weights_dicts
def _get_label_weights(y):
    #Get label weights for majority and minority class using the following:
        #major_weight=n/(n_major*2)
        #minor_weight=n*(n_major/n_minor)/(n_major*2)
        #NaN weights are set to zero
    y1=y[~np.isnan(y)]
    n=len(y1)
    n_zero=np.count_nonzero(np.isclose(y1,0))
    n_one=np.count_nonzero(np.isclose(y1,1))
    if n_zero>n_one:
        weight_zero=n/(n_zero*2)
        weight_one=n*(n_zero/n_one)/(n_zero*2)
    else:
        weight_zero=n*(n_one/n_zero)/(n_one*2)
        weight_one=n/(n_one*2)
    return weight_zero, weight_one    

class BinaryCrossEntropyIgnoreNaN(tf.keras.losses.Loss):
    def __init__(self, weights_dicts=None, axis=0, **kwargs):
        super(BinaryCrossEntropyIgnoreNaN, self).__init__(**kwargs)
        self.weights_dicts=weights_dicts
        self.axis=axis        

    def __call__(self, target, output, sample_weight=None):
        #Binary cross entropy that ignores Nan and replaces with mini-batch Nan with 0
        #modified from tf.python.keras.backend.binary_crossentropy
        
        ##NEED TO TEST THIS CODE MORE THOROUGHLY
        target=tf.convert_to_tensor(target)
        output=tf.convert_to_tensor(output)
        if len(target.shape)==1:
            target=tf.expand_dims(target, 1)
            output=tf.expand_dims(output, 1)
        epsilon_ = tf.constant(epsilon(), dtype=output.dtype.base_dtype)
        output=tf.clip_by_value(output, epsilon_, 1. - epsilon_)

        #Compute cross entropy from probabilities
        bce=target * tf.math.log(output+epsilon_)
        bce+=(1-target)* tf.math.log(1-output+epsilon_)

        bce=tf.where(tf.math.is_nan(-bce), epsilon(), -bce)
        if self.weights_dicts is not None:
            sample_weight=tf.cast(tf.where(target==0.,1.,0.)*[self.weights_dicts[i]['weight_zero'] for i in range(len(self.weights_dicts))], dtype=target.dtype)
            sample_weight+=tf.cast(tf.where(target==1., 1., 0.)*[self.weights_dicts[i]['weight_one'] for i in range(len(self.weights_dicts))], dtype=target.dtype)
            bce=tf.multiply(sample_weight, bce)
#         return tf.keras.backend.mean(bce, axis=self.axis)  
        return tf.math.reduce_mean(bce)

    def call(self, target, output, sample_weight=None):
        return self(target, output, sample_weight=None)

## Load Single label dataset

In [14]:
label='NR-AR'

train_inds = ~np.isnan(y_train[label])
test_inds = ~np.isnan(y_test[label])

train_data = X_train[train_inds, :]
train_targets = y_train[label][train_inds].astype(np.float32)
test_data = X_test[test_inds, :]
test_targets = y_test[label][test_inds].astype(np.float32)
train_Fweights = Fweights_train[train_inds,:]
test_Fweights = Fweights_test[test_inds,:]

train_tensor=np.hstack([train_data, train_Fweights])
test_tensor=np.hstack([test_data, test_Fweights])

## Model architecture

In [154]:
n_feat = train_data.shape[1]
n_attention = 20 #Reduced from 20 to 10. 10 works better
n_attention_hidden=20
n_attention_out=1
n_concat_hidden=2048
n_hidden1 =1024
n_hidden2 = 128 #Added 2nd hidden layer
momentum=0.8
learning_rate=0.001

n_batch=32



save_folder=os.path.join(time.strftime("%y%m%d_TrainingLocalitySensitivewFW",
                                       time.localtime()))
checkpoint_path = os.path.join(save_folder, 
                               "LocalitySensitivewFW_{}".format(label),
                               )

try: 
    os.mkdir(save_folder) 
except OSError as error: 
    print(error) 
    
try:
    os.mkdir(checkpoint_path)
except OSError as error:
    print(error)

from tensorflow.keras.initializers import VarianceScaling
concat_activation="gelu"
attention_hidden_activation="gelu"
attention_output_activation="sigmoid"
kernel_initializer=VarianceScaling()
hidden_activation="gelu"

[WinError 183] Cannot create a file when that file already exists: '210120_TrainingLocalitySensitivewFW'
[WinError 183] Cannot create a file when that file already exists: '210120_TrainingLocalitySensitivewFW\\LocalitySensitivewFW_NR-AR'


In [155]:
from algorithms import attention_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l1

np.random.seed(0)

input_layer=Input(shape=(n_feat*2, ))
attentions_layer=[attention_model.DenseAttentionwFeatWeights(
    n_feat,
    n_attention_hidden, 
    out=n_hidden_out,
    hidden_activation=attention_hidden_activation,
    output_activation=attention_output_activation,
    
)]

# attentions_layer=attention_model.ConcatAttentionswFeatWeights(
#     n_attention=n_attention,
#     n_attention_hidden=n_attention_hidden,
#     n_attention_out=n_attention_out,
#     n_feat=n_feat,
#     n_hidden=n_concat_hidden,
#     activation=concat_activation, 
#     kernel_initializer=kernel_initializer,
#     kernel_regularizer=l1(1E-5),
#     bias_regularizer=l1(1E-5),
#     attention_initializer=kernel_initializer,
#     attention_hidden_activation=attention_hidden_activation,
#     attention_output_activation=attention_output_activation
# )(input_layer)
dropout0=Dropout(0.1)(attentions_layer)
dense_layer1=Dense(n_hidden1, 
                   activation=hidden_activation, 
                   kernel_initializer=kernel_initializer,
                   kernel_regularizer=l1(1E-5),
                   bias_regularizer=l1(1E-5),
                  )(dropout0)
dropout1=Dropout(0.1)(dense_layer1)
dense_layer2=Dense(n_hidden2,
                   activation=hidden_activation,
                   kernel_initializer=kernel_initializer,
                   kernel_regularizer=l1(1E-5),
                   bias_regularizer=l1(1E-5)
                  )(dropout1)
dropout2=Dropout(0.1)(dense_layer2)
output_layer=Dense(1, activation='sigmoid')(dropout2)

LSwFW_model=Model(inputs=input_layer, 
                  outputs=output_layer
                 )

weights_dicts=get_weights_dicts(np.expand_dims(train_targets,1))
loss_fn=BinaryCrossEntropyIgnoreNaN(weights_dicts=weights_dicts)

LSwFW_model.compile(loss=loss_fn, 
              optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy', 'AUC']
             )

In [156]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_auc',
                                                 mode='max',
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)

csv_filename = os.path.join(checkpoint_path,
                            "training_log.csv"
                            )
csvlogger_callback = tf.keras.callbacks.CSVLogger(filename=csv_filename, append=True)


n_epoch=100


LSwFW_model.fit(train_tensor, 
                train_targets, 
                epochs=n_epoch,
                batch_size=n_batch,
                validation_data=(test_tensor, test_targets),
                shuffle=True,
                verbose=2, 
                callbacks=[csvlogger_callback,
                           cp_callback
                          ]
               )

Epoch 1/100
195/195 - 33s - loss: 8.6768 - accuracy: 0.9644 - auc: 0.4965 - val_loss: 7.9526 - val_accuracy: 0.9791 - val_auc: 0.5000

Epoch 00001: val_auc improved from -inf to 0.50000, saving model to 210120_TrainingLocalitySensitivewFW\LocalitySensitivewFW_NR-AR
Epoch 2/100
195/195 - 14s - loss: 2.0181 - accuracy: 0.6468 - auc: 0.6117 - val_loss: 1.2486 - val_accuracy: 0.4261 - val_auc: 0.6636

Epoch 00002: val_auc improved from 0.50000 to 0.66363, saving model to 210120_TrainingLocalitySensitivewFW\LocalitySensitivewFW_NR-AR
Epoch 3/100
195/195 - 14s - loss: 1.0373 - accuracy: 0.6927 - auc: 0.7375 - val_loss: 0.9501 - val_accuracy: 0.8713 - val_auc: 0.6890

Epoch 00003: val_auc improved from 0.66363 to 0.68902, saving model to 210120_TrainingLocalitySensitivewFW\LocalitySensitivewFW_NR-AR
Epoch 4/100
195/195 - 14s - loss: 0.8154 - accuracy: 0.7310 - auc: 0.7706 - val_loss: 0.9018 - val_accuracy: 0.9513 - val_auc: 0.6860

Epoch 00004: val_auc did not improve from 0.68902
Epoch 5/100


Epoch 00043: val_auc did not improve from 0.69664
Epoch 44/100
195/195 - 14s - loss: 0.3972 - accuracy: 0.8232 - auc: 0.9435 - val_loss: 2.2092 - val_accuracy: 0.8052 - val_auc: 0.6711

Epoch 00044: val_auc did not improve from 0.69664
Epoch 45/100
195/195 - 14s - loss: 0.4180 - accuracy: 0.8000 - auc: 0.9289 - val_loss: 1.9424 - val_accuracy: 0.7270 - val_auc: 0.6331

Epoch 00045: val_auc did not improve from 0.69664
Epoch 46/100
195/195 - 14s - loss: 0.5983 - accuracy: 0.7846 - auc: 0.9007 - val_loss: 2.9123 - val_accuracy: 0.9583 - val_auc: 0.6793

Epoch 00046: val_auc did not improve from 0.69664
Epoch 47/100
195/195 - 14s - loss: 0.5015 - accuracy: 0.8223 - auc: 0.9269 - val_loss: 1.9013 - val_accuracy: 0.7426 - val_auc: 0.6750

Epoch 00047: val_auc did not improve from 0.69664
Epoch 48/100
195/195 - 14s - loss: 0.4289 - accuracy: 0.8207 - auc: 0.9401 - val_loss: 2.7197 - val_accuracy: 0.8574 - val_auc: 0.6844

Epoch 00048: val_auc did not improve from 0.69664
Epoch 49/100
195/19

195/195 - 14s - loss: 0.3394 - accuracy: 0.8969 - auc: 0.9735 - val_loss: 2.7593 - val_accuracy: 0.8191 - val_auc: 0.6427

Epoch 00088: val_auc did not improve from 0.69664
Epoch 89/100
195/195 - 14s - loss: 0.3416 - accuracy: 0.8858 - auc: 0.9684 - val_loss: 3.3463 - val_accuracy: 0.9130 - val_auc: 0.6448

Epoch 00089: val_auc did not improve from 0.69664
Epoch 90/100
195/195 - 14s - loss: 0.2788 - accuracy: 0.9108 - auc: 0.9792 - val_loss: 3.4435 - val_accuracy: 0.8939 - val_auc: 0.6406

Epoch 00090: val_auc did not improve from 0.69664
Epoch 91/100
195/195 - 14s - loss: 0.2786 - accuracy: 0.9071 - auc: 0.9781 - val_loss: 3.5347 - val_accuracy: 0.9183 - val_auc: 0.6632

Epoch 00091: val_auc did not improve from 0.69664
Epoch 92/100
195/195 - 14s - loss: 0.3032 - accuracy: 0.8842 - auc: 0.9702 - val_loss: 4.5120 - val_accuracy: 0.9548 - val_auc: 0.5771

Epoch 00092: val_auc did not improve from 0.69664
Epoch 93/100
195/195 - 14s - loss: 0.3077 - accuracy: 0.8863 - auc: 0.9719 - val_lo

In [157]:
LSwFW_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 632)]             0         
_________________________________________________________________
concat_attentionsw_feat_weig (None, 8192)              51915172  
_________________________________________________________________
dropout_81 (Dropout)         (None, 8192)              0         
_________________________________________________________________
dense_148 (Dense)            (None, 1024)              8389632   
_________________________________________________________________
dropout_82 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_149 (Dense)            (None, 128)               131200    
_________________________________________________________________
dropout_83 (Dropout)         (None, 128)               0   

In [153]:
dense_model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_142 (Dense)            (None, 400)               126800    
_________________________________________________________________
dropout_77 (Dropout)         (None, 400)               0         
_________________________________________________________________
dense_143 (Dense)            (None, 6320)              2534320   
_________________________________________________________________
dense_144 (Dense)            (None, 2048)              12945408  
_________________________________________________________________
dropout_78 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_145 (Dense)            (None, 512)               1049088   
_________________________________________________________________
dropout_79 (Dropout)         (None, 512)             

## Loop training

In [7]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [59]:
n_feat = train_data.shape[1]
n_attention = 20 #Reduced from 20 to 10. 10 works better
n_attention_hidden=20
n_attention_out=16
n_concat_hidden=2048
n_hidden1 =1024
n_hidden2 = 128 #Added 2nd hidden layer
momentum=0.8
learning_rate=0.001

n_batch=8



from tensorflow.keras.initializers import VarianceScaling
concat_activation="gelu"
attention_hidden_activation="gelu"
attention_output_activation="sigmoid"
kernel_initializer=VarianceScaling()
hidden_activation="gelu"

In [60]:
from algorithms import attention_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.regularizers import l1

def initialize_LSwFWmodel(weights_dicts,
                     n_attention=n_attention, 
                     n_attention_hidden=n_attention_hidden, 
                     n_feat=n_feat,
                     n_concat_hidden=n_concat_hidden,
                     n_hidden1=n_hidden1, 
                     n_hidden2=n_hidden2,
                          n_hidden_out=n_attention_out, 
                          attention_hidden_activation=attention_hidden_activation,
                          attention_output_activation=attention_output_activation,
                          kernel_initializer=kernel_initializer,
                          hidden_activation=hidden_activation
                         ):

    input_layer=Input(shape=(n_feat*2, ))
    attention_heads_=[attention_model.DenseAttentionwFeatWeights(
        n_feat,
        n_attention_hidden, 
        out=n_hidden_out,
        hidden_activation=attention_hidden_activation,
        output_activation=attention_output_activation,
    )(input_layer) for i in range(n_attention)]
    concat_layer=Concatenate()(attention_heads_)
    # attentions_layer=attention_model.ConcatAttentionswFeatWeights(
    #     n_attention=n_attention,
    #     n_attention_hidden=n_attention_hidden,
    #     n_attention_out=n_attention_out,
    #     n_feat=n_feat,
    #     n_hidden=n_concat_hidden,
    #     activation=concat_activation, 
    #     kernel_initializer=kernel_initializer,
    #     kernel_regularizer=l1(1E-5),
    #     bias_regularizer=l1(1E-5),
    #     attention_initializer=kernel_initializer,
    #     attention_hidden_activation=attention_hidden_activation,
    #     attention_output_activation=attention_output_activation
    # )(input_layer)
    dropout0=Dropout(0.1)(concat_layer)
    dense_layer1=Dense(n_hidden1, 
                       activation=hidden_activation, 
                       kernel_initializer=kernel_initializer,
                       kernel_regularizer=l1(1E-5),
                       bias_regularizer=l1(1E-5),
                      )(dropout0)
    dropout1=Dropout(0.1)(dense_layer1)
    dense_layer2=Dense(n_hidden2,
                       activation=hidden_activation,
                       kernel_initializer=kernel_initializer,
                       kernel_regularizer=l1(1E-5),
                       bias_regularizer=l1(1E-5)
                      )(dropout1)
    dropout2=Dropout(0.1)(dense_layer2)
    output_layer=Dense(1, activation='sigmoid')(dropout2)

    LSwFW_model=Model(inputs=input_layer, 
                      outputs=output_layer
                     )

#     weights_dicts=get_weights_dicts(np.expand_dims(train_targets,1))
    loss_fn=BinaryCrossEntropyIgnoreNaN(weights_dicts=weights_dicts)

    LSwFW_model.compile(loss=loss_fn, 
                  optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['accuracy', 'AUC']
                 )
    return LSwFW_model

In [61]:
# Repeat above for all labels
n_epoch = 100
save_folder=os.path.join(time.strftime("%y%m%d_TrainingLocalitySensitivewFW",
                                       time.localtime()))
tmp_file = os.path.join(save_folder,
"{}_LSwFW_tmpfile.csv".format(time.strftime("%y%m%d", time.localtime()),
                     ))
try: 
    os.mkdir(save_folder) 
except OSError as error: 
    print(error)   
    
with open(tmp_file, 'w') as tmpfile:
    tmpfile.write(",".join(["Label", "Best_valAUC\n"]))

for label in y_train.columns:
    # Load data
    train_inds = ~np.isnan(y_train[label])
    test_inds = ~np.isnan(y_test[label])

    train_data = X_train[train_inds, :]
    train_targets = y_train[label][train_inds].astype(np.float32)
    test_data = X_test[test_inds, :]
    test_targets = y_test[label][test_inds].astype(np.float32)
    train_Fweights = Fweights_train[train_inds,:]
    test_Fweights = Fweights_test[test_inds,:]
    
    train_tensor=np.hstack([train_data, train_Fweights])
    test_tensor=np.hstack([test_data, test_Fweights])

    weights_dicts = get_weights_dicts(np.expand_dims(train_targets, 1))

    # Set save folders
    
    checkpoint_path = os.path.join(save_folder, "{}_LSwFW_{}".format(time.strftime("%y%m%d", time.localtime()),
                     label)
                                  )   

    try:
        os.mkdir(checkpoint_path)
    except OSError as error:
        print(error)

    np.random.seed(0)

    LSwFW_model_=initialize_LSwFWmodel(weights_dicts = weights_dicts)


    cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 monitor = 'val_auc',
                                                 mode = 'max',
                                                 save_best_only = True,
                                                 save_weights_only = True,
                                                 verbose = 1)

    csv_filename=os.path.join(checkpoint_path, "training_log.csv")
    csvlogger_callback=tf.keras.callbacks.CSVLogger(
        filename = csv_filename, append = True)

    LSwFW_model_.fit(train_tensor,
                    train_targets,
                    epochs=n_epoch, 
                    batch_size=n_batch, 
                    validation_data=(test_tensor, test_targets),
                    verbose=2,
                    shuffle=True,
                    callbacks=[csvlogger_callback,
                               cp_callback
                              ])
    
    with open(tmp_file, 'a') as tmpfile:
        tmpfile.write(",".join([label, str(cp_callback.best)+"/n"]))
    
    del LSwFW_model_

Epoch 1/100
1168/1168 - 130s - loss: 0.7179 - accuracy: 0.8371 - auc: 0.7473 - val_loss: 0.5867 - val_accuracy: 0.9513 - val_auc: 0.6661

Epoch 00001: val_auc improved from -inf to 0.66615, saving model to 210126_TrainingLocalitySensitivewFW\210126_LSwFW_NR-AR
Epoch 2/100
1168/1168 - 115s - loss: 0.5822 - accuracy: 0.9170 - auc: 0.8039 - val_loss: 0.5359 - val_accuracy: 0.9565 - val_auc: 0.6673

Epoch 00002: val_auc improved from 0.66615 to 0.66726, saving model to 210126_TrainingLocalitySensitivewFW\210126_LSwFW_NR-AR
Epoch 3/100
1168/1168 - 115s - loss: 0.5397 - accuracy: 0.9125 - auc: 0.8023 - val_loss: 0.5053 - val_accuracy: 0.9583 - val_auc: 0.6912

Epoch 00003: val_auc improved from 0.66726 to 0.69116, saving model to 210126_TrainingLocalitySensitivewFW\210126_LSwFW_NR-AR
Epoch 4/100
1168/1168 - 109s - loss: 0.5068 - accuracy: 0.9309 - auc: 0.8177 - val_loss: 0.4743 - val_accuracy: 0.9530 - val_auc: 0.6812

Epoch 00004: val_auc did not improve from 0.69116
Epoch 5/100
1168/1168 -


Epoch 00040: val_auc did not improve from 0.72077
Epoch 41/100
1168/1168 - 112s - loss: 0.3498 - accuracy: 0.8915 - auc: 0.9350 - val_loss: 0.7808 - val_accuracy: 0.9148 - val_auc: 0.7076

Epoch 00041: val_auc did not improve from 0.72077
Epoch 42/100
1168/1168 - 107s - loss: 0.3171 - accuracy: 0.8801 - auc: 0.9465 - val_loss: 1.0527 - val_accuracy: 0.8765 - val_auc: 0.7094

Epoch 00042: val_auc did not improve from 0.72077
Epoch 43/100
1168/1168 - 107s - loss: 0.3381 - accuracy: 0.8905 - auc: 0.9427 - val_loss: 0.9674 - val_accuracy: 0.7774 - val_auc: 0.6946

Epoch 00043: val_auc did not improve from 0.72077
Epoch 44/100
1168/1168 - 102s - loss: 0.3019 - accuracy: 0.8841 - auc: 0.9535 - val_loss: 0.7856 - val_accuracy: 0.8817 - val_auc: 0.6990

Epoch 00044: val_auc did not improve from 0.72077
Epoch 45/100
1168/1168 - 106s - loss: 0.3089 - accuracy: 0.8826 - auc: 0.9507 - val_loss: 0.8982 - val_accuracy: 0.8730 - val_auc: 0.6940

Epoch 00045: val_auc did not improve from 0.72077
Epoc


Epoch 00083: val_auc did not improve from 0.73816
Epoch 84/100
1168/1168 - 105s - loss: 0.1912 - accuracy: 0.9163 - auc: 0.9832 - val_loss: 1.4365 - val_accuracy: 0.9513 - val_auc: 0.6897

Epoch 00084: val_auc did not improve from 0.73816
Epoch 85/100
1168/1168 - 112s - loss: 0.2510 - accuracy: 0.9158 - auc: 0.9722 - val_loss: 1.5982 - val_accuracy: 0.9287 - val_auc: 0.7062

Epoch 00085: val_auc did not improve from 0.73816
Epoch 86/100
1168/1168 - 112s - loss: 0.1870 - accuracy: 0.9343 - auc: 0.9839 - val_loss: 1.5015 - val_accuracy: 0.9478 - val_auc: 0.7316

Epoch 00086: val_auc did not improve from 0.73816
Epoch 87/100
1168/1168 - 109s - loss: 0.2170 - accuracy: 0.9300 - auc: 0.9801 - val_loss: 1.9116 - val_accuracy: 0.9583 - val_auc: 0.6837

Epoch 00087: val_auc did not improve from 0.73816
Epoch 88/100
1168/1168 - 107s - loss: 0.2789 - accuracy: 0.9229 - auc: 0.9749 - val_loss: 1.6466 - val_accuracy: 0.9583 - val_auc: 0.7040

Epoch 00088: val_auc did not improve from 0.73816
Epoc


Epoch 00024: val_auc did not improve from 0.88779
Epoch 25/100
1025/1025 - 99s - loss: 0.3294 - accuracy: 0.8465 - auc: 0.9376 - val_loss: 0.5098 - val_accuracy: 0.6779 - val_auc: 0.8657

Epoch 00025: val_auc did not improve from 0.88779
Epoch 26/100
1025/1025 - 99s - loss: 0.3254 - accuracy: 0.8435 - auc: 0.9385 - val_loss: 0.5313 - val_accuracy: 0.6426 - val_auc: 0.8713

Epoch 00026: val_auc did not improve from 0.88779
Epoch 27/100
1025/1025 - 92s - loss: 0.3329 - accuracy: 0.8444 - auc: 0.9371 - val_loss: 0.5008 - val_accuracy: 0.7164 - val_auc: 0.8724

Epoch 00027: val_auc did not improve from 0.88779
Epoch 28/100
1025/1025 - 99s - loss: 0.3300 - accuracy: 0.8427 - auc: 0.9381 - val_loss: 0.5181 - val_accuracy: 0.6930 - val_auc: 0.8735

Epoch 00028: val_auc did not improve from 0.88779
Epoch 29/100
1025/1025 - 99s - loss: 0.3231 - accuracy: 0.8495 - auc: 0.9405 - val_loss: 0.5890 - val_accuracy: 0.8591 - val_auc: 0.8757

Epoch 00029: val_auc did not improve from 0.88779
Epoch 30/


Epoch 00068: val_auc did not improve from 0.88779
Epoch 69/100
1025/1025 - 95s - loss: 0.1989 - accuracy: 0.9113 - auc: 0.9778 - val_loss: 0.7192 - val_accuracy: 0.8289 - val_auc: 0.8687

Epoch 00069: val_auc did not improve from 0.88779
Epoch 70/100
1025/1025 - 94s - loss: 0.2037 - accuracy: 0.9087 - auc: 0.9769 - val_loss: 0.7210 - val_accuracy: 0.7685 - val_auc: 0.8709

Epoch 00070: val_auc did not improve from 0.88779
Epoch 71/100
1025/1025 - 94s - loss: 0.1925 - accuracy: 0.9161 - auc: 0.9786 - val_loss: 1.0055 - val_accuracy: 0.8507 - val_auc: 0.8470

Epoch 00071: val_auc did not improve from 0.88779
Epoch 72/100
1025/1025 - 88s - loss: 0.1976 - accuracy: 0.9207 - auc: 0.9783 - val_loss: 0.8144 - val_accuracy: 0.8490 - val_auc: 0.8563

Epoch 00072: val_auc did not improve from 0.88779
Epoch 73/100
1025/1025 - 94s - loss: 0.1851 - accuracy: 0.9273 - auc: 0.9804 - val_loss: 0.6959 - val_accuracy: 0.8171 - val_auc: 0.8668

Epoch 00073: val_auc did not improve from 0.88779
Epoch 74/

...

918/918 - 32s - loss: 0.3296 - accuracy: 0.8576 - auc: 0.9355 - val_loss: 0.3424 - val_accuracy: 0.8324 - val_auc: 0.9114

Epoch 00016: val_auc did not improve from 0.91797
Epoch 17/100
918/918 - 32s - loss: 0.3105 - accuracy: 0.8611 - auc: 0.9436 - val_loss: 0.4066 - val_accuracy: 0.8870 - val_auc: 0.9133

Epoch 00017: val_auc did not improve from 0.91797
Epoch 18/100
918/918 - 32s - loss: 0.3235 - accuracy: 0.8633 - auc: 0.9404 - val_loss: 0.4940 - val_accuracy: 0.6610 - val_auc: 0.9175

Epoch 00018: val_auc did not improve from 0.91797
Epoch 19/100
918/918 - 32s - loss: 0.3007 - accuracy: 0.8745 - auc: 0.9484 - val_loss: 0.3907 - val_accuracy: 0.7721 - val_auc: 0.9220

Epoch 00019: val_auc improved from 0.91797 to 0.92198, saving model to 210126_TrainingLocalitySensitivewFW\210127_LSwFW_SR-MMP
Epoch 20/100
918/918 - 32s - loss: 0.3007 - accuracy: 0.8686 - auc: 0.9475 - val_loss: 0.3273 - val_accuracy: 0.8324 - val_auc: 0.9164

Epoch 00020: val_auc did not improve from 0.92198
Epoch 


Epoch 00059: val_auc did not improve from 0.92198
Epoch 60/100
918/918 - 32s - loss: 0.1658 - accuracy: 0.9353 - auc: 0.9833 - val_loss: 0.4390 - val_accuracy: 0.8267 - val_auc: 0.8893

Epoch 00060: val_auc did not improve from 0.92198
Epoch 61/100
918/918 - 32s - loss: 0.1457 - accuracy: 0.9409 - auc: 0.9854 - val_loss: 0.5192 - val_accuracy: 0.8060 - val_auc: 0.8665

Epoch 00061: val_auc did not improve from 0.92198
Epoch 62/100
918/918 - 32s - loss: 0.1611 - accuracy: 0.9337 - auc: 0.9833 - val_loss: 0.7180 - val_accuracy: 0.8908 - val_auc: 0.8852

Epoch 00062: val_auc did not improve from 0.92198
Epoch 63/100
918/918 - 32s - loss: 0.1449 - accuracy: 0.9417 - auc: 0.9862 - val_loss: 0.7311 - val_accuracy: 0.8832 - val_auc: 0.8863

Epoch 00063: val_auc did not improve from 0.92198
Epoch 64/100
918/918 - 32s - loss: 0.1580 - accuracy: 0.9345 - auc: 0.9844 - val_loss: 0.6087 - val_accuracy: 0.8757 - val_auc: 0.8979

Epoch 00064: val_auc did not improve from 0.92198
Epoch 65/100
918/91

1081/1081 - 50s - loss: 0.6263 - accuracy: 0.6649 - auc: 0.7843 - val_loss: 0.6557 - val_accuracy: 0.4909 - val_auc: 0.8181

Epoch 00003: val_auc improved from 0.81004 to 0.81807, saving model to 210126_TrainingLocalitySensitivewFW\210127_LSwFW_SR-p53
Epoch 4/100
1081/1081 - 51s - loss: 0.5785 - accuracy: 0.6551 - auc: 0.7944 - val_loss: 0.5519 - val_accuracy: 0.5970 - val_auc: 0.8291

Epoch 00004: val_auc improved from 0.81807 to 0.82909, saving model to 210126_TrainingLocalitySensitivewFW\210127_LSwFW_SR-p53
Epoch 5/100
1081/1081 - 50s - loss: 0.5387 - accuracy: 0.6680 - auc: 0.8174 - val_loss: 0.5912 - val_accuracy: 0.4975 - val_auc: 0.8333

Epoch 00005: val_auc improved from 0.82909 to 0.83333, saving model to 210126_TrainingLocalitySensitivewFW\210127_LSwFW_SR-p53
Epoch 6/100
1081/1081 - 51s - loss: 0.5506 - accuracy: 0.6617 - auc: 0.8053 - val_loss: 0.5708 - val_accuracy: 0.5572 - val_auc: 0.8354

Epoch 00006: val_auc improved from 0.83333 to 0.83539, saving model to 210126_Train


Epoch 00043: val_auc did not improve from 0.84725
Epoch 44/100
1081/1081 - 53s - loss: 0.2717 - accuracy: 0.8701 - auc: 0.9574 - val_loss: 0.7838 - val_accuracy: 0.7894 - val_auc: 0.8385

Epoch 00044: val_auc did not improve from 0.84725
Epoch 45/100
1081/1081 - 53s - loss: 0.2700 - accuracy: 0.8705 - auc: 0.9588 - val_loss: 0.8627 - val_accuracy: 0.7512 - val_auc: 0.8313

Epoch 00045: val_auc did not improve from 0.84725
Epoch 46/100
1081/1081 - 52s - loss: 0.2747 - accuracy: 0.8841 - auc: 0.9606 - val_loss: 0.7859 - val_accuracy: 0.8441 - val_auc: 0.8441

Epoch 00046: val_auc did not improve from 0.84725
Epoch 47/100
1081/1081 - 52s - loss: 0.2800 - accuracy: 0.8673 - auc: 0.9566 - val_loss: 0.6299 - val_accuracy: 0.7678 - val_auc: 0.8086

Epoch 00047: val_auc did not improve from 0.84725
Epoch 48/100
1081/1081 - 52s - loss: 0.2547 - accuracy: 0.8776 - auc: 0.9638 - val_loss: 0.8683 - val_accuracy: 0.7993 - val_auc: 0.8423

Epoch 00048: val_auc did not improve from 0.84725
Epoch 49/


Epoch 00087: val_auc did not improve from 0.84725
Epoch 88/100
1081/1081 - 104s - loss: 0.1487 - accuracy: 0.9460 - auc: 0.9880 - val_loss: 1.6576 - val_accuracy: 0.8839 - val_auc: 0.8025

Epoch 00088: val_auc did not improve from 0.84725
Epoch 89/100
1081/1081 - 104s - loss: 0.1456 - accuracy: 0.9413 - auc: 0.9874 - val_loss: 2.5087 - val_accuracy: 0.9104 - val_auc: 0.7628

Epoch 00089: val_auc did not improve from 0.84725
Epoch 90/100
1081/1081 - 103s - loss: 0.1640 - accuracy: 0.9386 - auc: 0.9842 - val_loss: 1.1958 - val_accuracy: 0.8640 - val_auc: 0.7971

Epoch 00090: val_auc did not improve from 0.84725
Epoch 91/100
1081/1081 - 98s - loss: 0.1639 - accuracy: 0.9305 - auc: 0.9847 - val_loss: 0.9593 - val_accuracy: 0.8657 - val_auc: 0.8151

Epoch 00091: val_auc did not improve from 0.84725
Epoch 92/100
1081/1081 - 102s - loss: 0.1585 - accuracy: 0.9366 - auc: 0.9855 - val_loss: 1.4752 - val_accuracy: 0.8242 - val_auc: 0.7986

Epoch 00092: val_auc did not improve from 0.84725
Epoch

In [19]:
print(tmpfile)

<_io.TextIOWrapper name='210128_TrainingLocalitySensitivewoFW\\210128_LS_tmpfile.csv' mode='a' encoding='cp1252'>


In [65]:
label='NR-PPAR-gamma'

#File names
load_folder="210126_TrainingLocalitySensitivewFW"
checkpoint_path=os.path.join(load_folder, 
                             "210127_LSwFW_NR-PPAR-gamma"
                            )

#Get data
train_inds = ~np.isnan(y_train[label])
test_inds = ~np.isnan(y_test[label])

train_data = X_train[train_inds, :]
train_targets = y_train[label][train_inds].astype(np.float32)
test_data = X_test[test_inds, :]
test_targets = y_test[label][test_inds].astype(np.float32)
train_Fweights = Fweights_train[train_inds,:]
test_Fweights = Fweights_test[test_inds,:]

train_tensor=np.hstack([train_data, train_Fweights])
test_tensor=np.hstack([test_data, test_Fweights])

weights_dicts = get_weights_dicts(np.expand_dims(train_targets, 1))

#Initialize model
LSwFW_PPARg=initialize_LSwFWmodel(weights_dicts)

#Load model weights
LSwFW_PPARg.load_weights(checkpoint_path)

In [69]:
LSwFW_PPARg.evaluate(test_tensor, test_targets)

19/19 [==============================] - 1s 43ms/step - loss: 1.3769 - accuracy: 0.7504 - auc: 0.7902


[1.376907467842102, 0.7504215836524963, 0.7902308702468872]

# Load PPARg

In [81]:
dataset_name="ATG_PPARg_CDKPaDEL_processed.csv"
df=pd.read_csv(os.path.join("..", 
                            "..",
                            "rawdata", 
                            dataset_name), 
               index_col=0
              )

In [83]:
X_ATG=df.iloc[:,3:]
y_ATG=df.iloc[:,1]

In [ ]:
from algorithms import COSA
from importlib import reload
reload(COSA)
cosa_mdl=COSA.NNCosa(Fweight_init="uniform", lam=0.2, n_iter=1,
                                distance_measure="inv_exp_dist",
                     calc_D_ijk=False, threads=3
                    )
cosa_mdl.fit(X_train)

# Train Locality-Sensitive Model without Feature Weighting

In [177]:
label='NR-AR'

train_ind=~np.isnan(y_train_ARE[label])
test_ind=~np.isnan(y_test[label])
train_targets=y_train_ARE[label][train_ind]
test_targets=y_test_ARE[label][test_ind]

# train_tensor=np.hstack([train_data, train_Fweights
#                        ])[train_ind]
# test_tensor=np.hstack([test_data, test_Fweights
#                       ])[test_ind]

train_tensor=train_data[train_ind]
test_tensor=test_data[test_ind]



In [ ]:
n_feat = train_tensor.shape[1]
n_attention = 20 #Reduced from 20 to 10. 10 works better
n_attention_hidden=20
n_attention_out=1
n_concat_hidden=2048
n_hidden1 =1024
n_hidden2 = 128 #Added 2nd hidden layer
momentum=0.8
learning_rate=0.001

n_batch=32

save_folder=os.path.join(time.strftime("%y%m%d_TrainingLocalitySensitivewoFW",
                                       time.localtime()))
checkpoint_path = os.path.join(save_folder, 
                               "LocalitySensitivewoFW_{}".format(label),
                               )

try: 
    os.mkdir(save_folder) 
except OSError as error: 
    print(error) 
    
try:
    os.mkdir(checkpoint_path)
except OSError as error:
    print(error)

from tensorflow.keras.initializers import VarianceScaling
concat_activation="gelu"
attention_hidden_activation="gelu"
attention_output_activation="sigmoid"
kernel_initializer=VarianceScaling()
hidden_activation="gelu"

In [170]:
from algorithms import attention_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l1

np.random.seed(0)

input_layer=Input(shape=(n_feat, ))
attentions_layer=attention_model.ConcatAttentions(
    n_attention=n_attention,
    n_attention_hidden=n_attention_hidden,
    n_attention_out=n_attention_out,
    n_feat=n_feat,
    n_hidden=n_concat_hidden,
    activation=concat_activation,
    kernel_initializer=kernel_initializer,
    kernel_regularizer=l1(1E-5),
    bias_regularizer=l1(1E-5),
    attention_initializer=kernel_initializer,
    attention_hidden_activation=attention_hidden_activation,
    attention_output_activation=attention_output_activation
)(input_layer)

# attentions_layer=attention_model.ConcatAttentionswFeatWeights(
#     n_attention=n_attention,
#     n_attention_hidden=n_attention_hidden,
#     n_attention_out=n_attention_out,
#     n_feat=n_feat,
#     n_hidden=n_concat_hidden,
#     activation=concat_activation, 
#     kernel_initializer=kernel_initializer,
#     kernel_regularizer=l1(1E-5),
#     bias_regularizer=l1(1E-5),
#     attention_initializer=kernel_initializer,
#     attention_hidden_activation=attention_hidden_activation,
#     attention_output_activation=attention_output_activation
# )(input_layer)
dropout0=Dropout(0.1)(attentions_layer)
dense_layer1=Dense(n_hidden1, 
                   activation=hidden_activation, 
                   kernel_initializer=kernel_initializer,
                   kernel_regularizer=l1(1E-5),
                   bias_regularizer=l1(1E-5),
                  )(dropout0)
dropout1=Dropout(0.1)(dense_layer1)
dense_layer2=Dense(n_hidden2,
                   activation=hidden_activation,
                   kernel_initializer=kernel_initializer,
                   kernel_regularizer=l1(1E-5),
                   bias_regularizer=l1(1E-5)
                  )(dropout1)
dropout2=Dropout(0.1)(dense_layer2)
output_layer=Dense(1, activation='sigmoid')(dropout2)

LS_model=Model(inputs=input_layer, 
                  outputs=output_layer
                 )

weights_dicts=get_weights_dicts(np.expand_dims(train_targets,1))
loss_fn=BinaryCrossEntropyIgnoreNaN(weights_dicts=weights_dicts)

LS_model.compile(loss=loss_fn, 
              optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy', 'AUC']
             )

In [172]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_auc',
                                                 mode='max',
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)

csv_filename = os.path.join(checkpoint_path,
                            "training_log.csv"
                            )
csvlogger_callback = tf.keras.callbacks.CSVLogger(filename=csv_filename, append=True)


n_epoch=100


LS_model.fit(train_tensor, 
                train_targets, 
                epochs=n_epoch,
                batch_size=n_batch,
                validation_data=(test_tensor, test_targets),
                shuffle=True,
                verbose=2, 
                callbacks=[csvlogger_callback,
                           cp_callback
                          ]
               )

Epoch 1/100
195/195 - 20s - loss: 4.3347 - accuracy: 0.7292 - auc: 0.5129 - val_loss: 1.1057 - val_accuracy: 0.4817 - val_auc: 0.7046

Epoch 00001: val_auc improved from -inf to 0.70463, saving model to 210120_TrainingLocalitySensitivewFW\LocalitySensitivewoFW_NR-AR
Epoch 2/100
195/195 - 10s - loss: 0.9603 - accuracy: 0.6769 - auc: 0.7162 - val_loss: 0.8231 - val_accuracy: 0.6087 - val_auc: 0.7250

Epoch 00002: val_auc improved from 0.70463 to 0.72499, saving model to 210120_TrainingLocalitySensitivewFW\LocalitySensitivewoFW_NR-AR
Epoch 3/100
195/195 - 9s - loss: 0.7575 - accuracy: 0.7761 - auc: 0.7628 - val_loss: 0.7304 - val_accuracy: 0.6626 - val_auc: 0.7297

Epoch 00003: val_auc improved from 0.72499 to 0.72972, saving model to 210120_TrainingLocalitySensitivewFW\LocalitySensitivewoFW_NR-AR
Epoch 4/100
195/195 - 9s - loss: 0.6759 - accuracy: 0.7879 - auc: 0.7875 - val_loss: 0.7385 - val_accuracy: 0.9478 - val_auc: 0.7216

Epoch 00004: val_auc did not improve from 0.72972
Epoch 5/10

195/195 - 9s - loss: 0.3615 - accuracy: 0.8673 - auc: 0.9535 - val_loss: 3.0091 - val_accuracy: 0.8991 - val_auc: 0.6126

Epoch 00044: val_auc did not improve from 0.72972
Epoch 45/100
195/195 - 9s - loss: 0.3346 - accuracy: 0.8829 - auc: 0.9615 - val_loss: 2.6276 - val_accuracy: 0.8470 - val_auc: 0.6567

Epoch 00045: val_auc did not improve from 0.72972
Epoch 46/100
195/195 - 9s - loss: 0.3342 - accuracy: 0.8765 - auc: 0.9611 - val_loss: 3.1609 - val_accuracy: 0.9217 - val_auc: 0.6368

Epoch 00046: val_auc did not improve from 0.72972
Epoch 47/100
195/195 - 9s - loss: 0.3643 - accuracy: 0.8810 - auc: 0.9533 - val_loss: 2.7434 - val_accuracy: 0.8296 - val_auc: 0.6078

Epoch 00047: val_auc did not improve from 0.72972
Epoch 48/100
195/195 - 9s - loss: 0.3316 - accuracy: 0.8771 - auc: 0.9647 - val_loss: 2.9363 - val_accuracy: 0.8783 - val_auc: 0.6630

Epoch 00048: val_auc did not improve from 0.72972
Epoch 49/100
195/195 - 10s - loss: 0.3524 - accuracy: 0.8763 - auc: 0.9602 - val_loss: 3


Epoch 00088: val_auc did not improve from 0.72972
Epoch 89/100
195/195 - 10s - loss: 0.2295 - accuracy: 0.9320 - auc: 0.9868 - val_loss: 4.1075 - val_accuracy: 0.9096 - val_auc: 0.6317

Epoch 00089: val_auc did not improve from 0.72972
Epoch 90/100
195/195 - 9s - loss: 0.3439 - accuracy: 0.8842 - auc: 0.9689 - val_loss: 3.5172 - val_accuracy: 0.9165 - val_auc: 0.6472

Epoch 00090: val_auc did not improve from 0.72972
Epoch 91/100
195/195 - 10s - loss: 0.2451 - accuracy: 0.9259 - auc: 0.9853 - val_loss: 4.6613 - val_accuracy: 0.9183 - val_auc: 0.6328

Epoch 00091: val_auc did not improve from 0.72972
Epoch 92/100
195/195 - 9s - loss: 0.2138 - accuracy: 0.9304 - auc: 0.9893 - val_loss: 4.6815 - val_accuracy: 0.9235 - val_auc: 0.6044

Epoch 00092: val_auc did not improve from 0.72972
Epoch 93/100
195/195 - 10s - loss: 0.2248 - accuracy: 0.9302 - auc: 0.9877 - val_loss: 3.9253 - val_accuracy: 0.8261 - val_auc: 0.5850

Epoch 00093: val_auc did not improve from 0.72972
Epoch 94/100
195/195 

## Loop training for all single-labels

In [4]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler(with_mean=True, with_std=True)
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)


In [10]:
n_feat = X_train.shape[1]
n_attention = 10 #Reduced from 20 to 10. 10 works better
n_attention_hidden=20
n_attention_out=1
n_concat_hidden=2048
n_hidden1 =1024
n_hidden2 = 128 #Added 2nd hidden layer
learning_rate=0.001

n_batch=32

from tensorflow.keras.initializers import VarianceScaling
concat_activation="gelu"
attention_hidden_activation="gelu"
attention_output_activation="sigmoid"
kernel_initializer="glorot_uniform"
hidden_activation="gelu"

In [11]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [12]:
#Updated 6th Jan 2021 (Edited Line 71 to Line 72. Reduce_mean instead of mean, to preserve the required rank)

from tensorflow.keras import backend as K
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
epsilon=K.epsilon

def get_weights_dicts(Y):
    weights_dicts=[]
    for j in range(Y.shape[1]):
        weight_zero, weight_one = _get_label_weights(Y[:,j])
        d={'weight_zero':weight_zero,
           'weight_one':weight_one
          }
        weights_dicts.append(d)
    return weights_dicts
def _get_label_weights(y):
    #Get label weights for majority and minority class using the following:
        #major_weight=n/(n_major*2)
        #minor_weight=n*(n_major/n_minor)/(n_major*2)
        #NaN weights are set to zero
    y1=y[~np.isnan(y)]
    n=len(y1)
    n_zero=np.count_nonzero(np.isclose(y1,0))
    n_one=np.count_nonzero(np.isclose(y1,1))
    if n_zero>n_one:
        weight_zero=n/(n_zero*2)
        weight_one=n*(n_zero/n_one)/(n_zero*2)
    else:
        weight_zero=n*(n_one/n_zero)/(n_one*2)
        weight_one=n/(n_one*2)
    return weight_zero, weight_one    

class BinaryCrossEntropyIgnoreNaN(tf.keras.losses.Loss):
    def __init__(self, weights_dicts=None, axis=0, **kwargs):
        super(BinaryCrossEntropyIgnoreNaN, self).__init__(**kwargs)
        self.weights_dicts=weights_dicts
        self.axis=axis        

    def __call__(self, target, output, sample_weight=None):
        #Binary cross entropy that ignores Nan and replaces with mini-batch Nan with 0
        #modified from tf.python.keras.backend.binary_crossentropy
        
        ##NEED TO TEST THIS CODE MORE THOROUGHLY
        target=tf.convert_to_tensor(target)
        output=tf.convert_to_tensor(output)
        if len(target.shape)==1:
            target=tf.expand_dims(target, 1)
            output=tf.expand_dims(output, 1)
        epsilon_ = tf.constant(epsilon(), dtype=output.dtype.base_dtype)
        output=tf.clip_by_value(output, epsilon_, 1. - epsilon_)

        #Compute cross entropy from probabilities
        bce=target * tf.math.log(output+epsilon_)
        bce+=(1-target)* tf.math.log(1-output+epsilon_)

        bce=tf.where(tf.math.is_nan(-bce), epsilon(), -bce)
        if self.weights_dicts is not None:
            sample_weight=tf.cast(tf.where(target==0.,1.,0.)*[self.weights_dicts[i]['weight_zero'] for i in range(len(self.weights_dicts))], dtype=target.dtype)
            sample_weight+=tf.cast(tf.where(target==1., 1., 0.)*[self.weights_dicts[i]['weight_one'] for i in range(len(self.weights_dicts))], dtype=target.dtype)
            bce=tf.multiply(sample_weight, bce)
#         return tf.keras.backend.mean(bce, axis=self.axis)  
        return tf.math.reduce_mean(bce)

    def call(self, target, output, sample_weight=None):
        return self(target, output, sample_weight=None)

In [13]:
from algorithms import attention_model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1

def initialize_LSmodel(weights_dicts,
                     n_attention=n_attention, 
                     n_attention_hidden=n_attention_hidden, 
                     n_feat=n_feat,
                     n_concat_hidden=n_concat_hidden,
                     n_hidden1=n_hidden1, 
                     n_hidden2=n_hidden2,
                      ):
    input_layer=Input(shape=(n_feat, ))
    attentions_layer=attention_model.ConcatAttentions(
        n_attention=n_attention,
        n_attention_hidden=n_attention_hidden,
        n_attention_out=n_attention_out,
        n_feat=n_feat,
        n_hidden=n_concat_hidden,
        activation=concat_activation,
        kernel_initializer=kernel_initializer,
        kernel_regularizer=l1(1E-5),
        bias_regularizer=l1(1E-5),
        attention_initializer=kernel_initializer,
        attention_hidden_activation=attention_hidden_activation,
        attention_output_activation=attention_output_activation
    )(input_layer)

    # attentions_layer=attention_model.ConcatAttentionswFeatWeights(
    #     n_attention=n_attention,
    #     n_attention_hidden=n_attention_hidden,
    #     n_attention_out=n_attention_out,
    #     n_feat=n_feat,
    #     n_hidden=n_concat_hidden,
    #     activation=concat_activation, 
    #     kernel_initializer=kernel_initializer,
    #     kernel_regularizer=l1(1E-5),
    #     bias_regularizer=l1(1E-5),
    #     attention_initializer=kernel_initializer,
    #     attention_hidden_activation=attention_hidden_activation,
    #     attention_output_activation=attention_output_activation
    # )(input_layer)
    dropout0=Dropout(0.1)(attentions_layer)
    dense_layer1=Dense(n_hidden1, 
                       activation=hidden_activation, 
                       kernel_initializer=kernel_initializer,
                       kernel_regularizer=l1(1E-5),
                       bias_regularizer=l1(1E-5),
                      )(dropout0)
    dropout1=Dropout(0.1)(dense_layer1)
    dense_layer2=Dense(n_hidden2,
                       activation=hidden_activation,
                       kernel_initializer=kernel_initializer,
                       kernel_regularizer=l1(1E-5),
                       bias_regularizer=l1(1E-5)
                      )(dropout1)
    dropout2=Dropout(0.1)(dense_layer2)
    output_layer=Dense(1, activation='sigmoid')(dropout2)

    LS_model=Model(inputs=input_layer, 
                      outputs=output_layer
                     )

    weights_dicts=get_weights_dicts(np.expand_dims(train_targets,1))
    loss_fn=BinaryCrossEntropyIgnoreNaN(weights_dicts=weights_dicts)

    LS_model.compile(loss=loss_fn, 
#                   optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
                      optimizer=tf.keras.optimizers.Adagrad(
                          learning_rate=learning_rate
                      ),
                     metrics=['accuracy', 'AUC']
                 )
    
    return LS_model

In [14]:
# Repeat above for all labels
n_epoch = 100

save_folder=os.path.join(time.strftime("%y%m%d_TrainingLocalitySensitivewoFW",
                                       time.localtime()))
try: 
    os.mkdir(save_folder) 
except OSError as error: 
    print(error) 

tmp_file = os.path.join(save_folder,
"{}_LS_tmpfile.csv".format(time.strftime("%y%m%d", time.localtime()),
                     ))

with open(tmp_file, 'w') as tmpfile:
    tmpfile.write(",".join(["Label", "Best_AUC\n"]))

    
for label in y_train.columns:
    # Load data
    train_ind=~np.isnan(y_train[label])
    test_ind=~np.isnan(y_test[label])
    train_targets=y_train[label][train_ind]
    test_targets=y_test[label][test_ind]

    # train_tensor=np.hstack([train_data, train_Fweights
    #                        ])[train_ind]
    # test_tensor=np.hstack([test_data, test_Fweights
    #                       ])[test_ind]

    train_tensor=X_train_scaled[train_ind]
    test_tensor=X_test_scaled[test_ind]
    
    # train_Fweights
    # test_Fweights

    weights_dicts = get_weights_dicts(np.expand_dims(train_targets, 1))

    # Set save folders
    checkpoint_path = os.path.join(save_folder, "{}_LS_{}".format(time.strftime("%y%m%d", time.localtime()),
                     label)
                                  )
    try:
        os.mkdir(checkpoint_path)
    except OSError as error:
        print(error)

    np.random.seed(0)

    LS_model_=initialize_LSmodel(weights_dicts = weights_dicts)


    cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 monitor = 'val_auc',
                                                 mode = 'max',
                                                 save_best_only = True,
                                                 save_weights_only = True,
                                                 verbose = 1)

    csv_filename=os.path.join(checkpoint_path, "training_log.csv")
    csvlogger_callback=tf.keras.callbacks.CSVLogger(
        filename = csv_filename, append = True)

    LS_model_.fit(train_tensor,
                    train_targets,
                    epochs=n_epoch, 
                    batch_size=n_batch, 
                    validation_data=(test_tensor, test_targets),
                    verbose=2,
                    shuffle=True,
                    callbacks=[csvlogger_callback,
                               cp_callback
                              ])
    
    with open(tmp_file, 'a') as tmpfile:
        tmpfile.write(",".join([label, str(cp_callback.best)+"/n"]))
    
    del LS_model_

[WinError 183] Cannot create a file when that file already exists: '210128_TrainingLocalitySensitivewoFW'
[WinError 183] Cannot create a file when that file already exists: '210128_TrainingLocalitySensitivewoFW\\210128_LS_NR-AR'
Epoch 1/100
292/292 - 18s - loss: 3.2246 - accuracy: 0.8007 - auc: 0.8165 - val_loss: 3.1682 - val_accuracy: 0.8591 - val_auc: 0.7234

Epoch 00001: val_auc improved from -inf to 0.72343, saving model to 210128_TrainingLocalitySensitivewoFW\210128_LS_NR-AR
Epoch 2/100
292/292 - 14s - loss: 3.1270 - accuracy: 0.9002 - auc: 0.8819 - val_loss: 3.1935 - val_accuracy: 0.8400 - val_auc: 0.7157

Epoch 00002: val_auc did not improve from 0.72343
Epoch 3/100
292/292 - 14s - loss: 3.0876 - accuracy: 0.9059 - auc: 0.9134 - val_loss: 3.1910 - val_accuracy: 0.8852 - val_auc: 0.7317

Epoch 00003: val_auc improved from 0.72343 to 0.73172, saving model to 210128_TrainingLocalitySensitivewoFW\210128_LS_NR-AR
Epoch 4/100
292/292 - 14s - loss: 3.0639 - accuracy: 0.9078 - auc: 0.92


Epoch 00043: val_auc did not improve from 0.73172
Epoch 44/100
292/292 - 13s - loss: 2.6916 - accuracy: 0.9785 - auc: 0.9972 - val_loss: 4.3772 - val_accuracy: 0.9548 - val_auc: 0.6112

Epoch 00044: val_auc did not improve from 0.73172
Epoch 45/100
292/292 - 13s - loss: 2.6871 - accuracy: 0.9802 - auc: 0.9973 - val_loss: 4.4767 - val_accuracy: 0.9635 - val_auc: 0.6067

Epoch 00045: val_auc did not improve from 0.73172
Epoch 46/100
292/292 - 13s - loss: 2.6857 - accuracy: 0.9809 - auc: 0.9975 - val_loss: 4.3715 - val_accuracy: 0.9565 - val_auc: 0.6214

Epoch 00046: val_auc did not improve from 0.73172
Epoch 47/100
292/292 - 13s - loss: 2.6883 - accuracy: 0.9793 - auc: 0.9968 - val_loss: 4.4750 - val_accuracy: 0.9600 - val_auc: 0.6078

Epoch 00047: val_auc did not improve from 0.73172
Epoch 48/100
292/292 - 14s - loss: 2.6814 - accuracy: 0.9810 - auc: 0.9973 - val_loss: 4.4958 - val_accuracy: 0.9652 - val_auc: 0.6137

Epoch 00048: val_auc did not improve from 0.73172
Epoch 49/100
292/29

292/292 - 14s - loss: 2.5968 - accuracy: 0.9861 - auc: 0.9983 - val_loss: 4.7453 - val_accuracy: 0.9652 - val_auc: 0.5684

Epoch 00088: val_auc did not improve from 0.73172
Epoch 89/100
292/292 - 14s - loss: 2.5939 - accuracy: 0.9862 - auc: 0.9983 - val_loss: 4.8117 - val_accuracy: 0.9687 - val_auc: 0.5767

Epoch 00089: val_auc did not improve from 0.73172
Epoch 90/100
292/292 - 14s - loss: 2.5945 - accuracy: 0.9863 - auc: 0.9982 - val_loss: 4.7867 - val_accuracy: 0.9670 - val_auc: 0.5713

Epoch 00090: val_auc did not improve from 0.73172
Epoch 91/100
292/292 - 13s - loss: 2.5919 - accuracy: 0.9865 - auc: 0.9982 - val_loss: 4.7439 - val_accuracy: 0.9617 - val_auc: 0.5741

Epoch 00091: val_auc did not improve from 0.73172
Epoch 92/100
292/292 - 13s - loss: 2.5934 - accuracy: 0.9851 - auc: 0.9981 - val_loss: 4.6579 - val_accuracy: 0.9583 - val_auc: 0.5633

Epoch 00092: val_auc did not improve from 0.73172
Epoch 93/100
292/292 - 13s - loss: 2.5884 - accuracy: 0.9866 - auc: 0.9980 - val_lo

257/257 - 12s - loss: 2.7808 - accuracy: 0.9594 - auc: 0.9927 - val_loss: 3.4992 - val_accuracy: 0.8708 - val_auc: 0.8845

Epoch 00028: val_auc did not improve from 0.89650
Epoch 29/100
257/257 - 12s - loss: 2.7762 - accuracy: 0.9592 - auc: 0.9934 - val_loss: 3.5398 - val_accuracy: 0.8859 - val_auc: 0.8833

Epoch 00029: val_auc did not improve from 0.89650
Epoch 30/100
257/257 - 12s - loss: 2.7734 - accuracy: 0.9602 - auc: 0.9933 - val_loss: 3.4732 - val_accuracy: 0.8775 - val_auc: 0.8909

Epoch 00030: val_auc did not improve from 0.89650
Epoch 31/100
257/257 - 10s - loss: 2.7693 - accuracy: 0.9605 - auc: 0.9935 - val_loss: 3.5163 - val_accuracy: 0.8742 - val_auc: 0.8838

Epoch 00031: val_auc did not improve from 0.89650
Epoch 32/100
257/257 - 11s - loss: 2.7594 - accuracy: 0.9641 - auc: 0.9942 - val_loss: 3.5147 - val_accuracy: 0.8826 - val_auc: 0.8848

Epoch 00032: val_auc did not improve from 0.89650
Epoch 33/100
257/257 - 11s - loss: 2.7561 - accuracy: 0.9653 - auc: 0.9946 - val_lo


Epoch 00072: val_auc did not improve from 0.89650
Epoch 73/100
257/257 - 11s - loss: 2.6386 - accuracy: 0.9868 - auc: 0.9989 - val_loss: 4.0098 - val_accuracy: 0.8960 - val_auc: 0.8326

Epoch 00073: val_auc did not improve from 0.89650
Epoch 74/100
257/257 - 10s - loss: 2.6398 - accuracy: 0.9855 - auc: 0.9984 - val_loss: 3.9248 - val_accuracy: 0.8926 - val_auc: 0.8529

Epoch 00074: val_auc did not improve from 0.89650
Epoch 75/100
257/257 - 10s - loss: 2.6345 - accuracy: 0.9876 - auc: 0.9986 - val_loss: 4.3747 - val_accuracy: 0.9010 - val_auc: 0.8137

Epoch 00075: val_auc did not improve from 0.89650
Epoch 76/100
257/257 - 10s - loss: 2.6368 - accuracy: 0.9857 - auc: 0.9985 - val_loss: 3.9939 - val_accuracy: 0.8960 - val_auc: 0.8488

Epoch 00076: val_auc did not improve from 0.89650
Epoch 77/100
257/257 - 10s - loss: 2.6338 - accuracy: 0.9862 - auc: 0.9986 - val_loss: 3.9084 - val_accuracy: 0.8960 - val_auc: 0.8531

Epoch 00077: val_auc did not improve from 0.89650
Epoch 78/100
257/25

...

257/257 - 12s - loss: 2.5940 - accuracy: 0.9892 - auc: 0.9989 - val_loss: 4.5593 - val_accuracy: 0.9413 - val_auc: 0.6945

Epoch 00097: val_auc did not improve from 0.76494
Epoch 98/100
257/257 - 12s - loss: 2.5915 - accuracy: 0.9898 - auc: 0.9989 - val_loss: 4.6327 - val_accuracy: 0.9413 - val_auc: 0.6958

Epoch 00098: val_auc did not improve from 0.76494
Epoch 99/100
257/257 - 12s - loss: 2.5910 - accuracy: 0.9897 - auc: 0.9986 - val_loss: 4.6482 - val_accuracy: 0.9430 - val_auc: 0.6705

Epoch 00099: val_auc did not improve from 0.76494
Epoch 100/100
257/257 - 12s - loss: 2.5893 - accuracy: 0.9878 - auc: 0.9989 - val_loss: 4.6667 - val_accuracy: 0.9446 - val_auc: 0.6766

Epoch 00100: val_auc did not improve from 0.76494
Epoch 1/100
230/230 - 14s - loss: 3.2523 - accuracy: 0.7003 - auc: 0.8253 - val_loss: 3.1149 - val_accuracy: 0.7665 - val_auc: 0.9029

Epoch 00001: val_auc improved from -inf to 0.90295, saving model to 210128_TrainingLocalitySensitivewoFW\210129_LS_SR-MMP
Epoch 2/100


Epoch 00037: val_auc did not improve from 0.93581
Epoch 38/100
230/230 - 11s - loss: 2.7404 - accuracy: 0.9717 - auc: 0.9961 - val_loss: 3.1386 - val_accuracy: 0.9021 - val_auc: 0.9160

Epoch 00038: val_auc did not improve from 0.93581
Epoch 39/100
230/230 - 11s - loss: 2.7298 - accuracy: 0.9744 - auc: 0.9970 - val_loss: 3.1307 - val_accuracy: 0.9002 - val_auc: 0.9193

Epoch 00039: val_auc did not improve from 0.93581
Epoch 40/100
230/230 - 11s - loss: 2.7288 - accuracy: 0.9747 - auc: 0.9968 - val_loss: 3.1063 - val_accuracy: 0.9058 - val_auc: 0.9288

Epoch 00040: val_auc did not improve from 0.93581
Epoch 41/100
230/230 - 11s - loss: 2.7266 - accuracy: 0.9744 - auc: 0.9969 - val_loss: 3.1264 - val_accuracy: 0.9021 - val_auc: 0.9158

Epoch 00041: val_auc did not improve from 0.93581
Epoch 42/100
230/230 - 11s - loss: 2.7202 - accuracy: 0.9773 - auc: 0.9972 - val_loss: 3.1108 - val_accuracy: 0.9077 - val_auc: 0.9257

Epoch 00042: val_auc did not improve from 0.93581
Epoch 43/100
230/23

230/230 - 10s - loss: 2.6361 - accuracy: 0.9879 - auc: 0.9985 - val_loss: 3.2892 - val_accuracy: 0.9040 - val_auc: 0.8959

Epoch 00082: val_auc did not improve from 0.93581
Epoch 83/100
230/230 - 11s - loss: 2.6323 - accuracy: 0.9890 - auc: 0.9990 - val_loss: 3.2685 - val_accuracy: 0.9096 - val_auc: 0.8945

Epoch 00083: val_auc did not improve from 0.93581
Epoch 84/100
230/230 - 11s - loss: 2.6323 - accuracy: 0.9879 - auc: 0.9989 - val_loss: 3.2608 - val_accuracy: 0.9077 - val_auc: 0.9002

Epoch 00084: val_auc did not improve from 0.93581
Epoch 85/100
230/230 - 11s - loss: 2.6279 - accuracy: 0.9880 - auc: 0.9990 - val_loss: 3.2494 - val_accuracy: 0.9021 - val_auc: 0.8946

Epoch 00085: val_auc did not improve from 0.93581
Epoch 86/100
230/230 - 10s - loss: 2.6271 - accuracy: 0.9883 - auc: 0.9991 - val_loss: 3.2574 - val_accuracy: 0.9134 - val_auc: 0.9012

Epoch 00086: val_auc did not improve from 0.93581
Epoch 87/100
230/230 - 11s - loss: 2.6272 - accuracy: 0.9883 - auc: 0.9989 - val_lo


Epoch 00023: val_auc did not improve from 0.84694
Epoch 24/100
271/271 - 12s - loss: 2.7383 - accuracy: 0.9794 - auc: 0.9975 - val_loss: 3.8623 - val_accuracy: 0.8789 - val_auc: 0.8218

Epoch 00024: val_auc did not improve from 0.84694
Epoch 25/100
271/271 - 12s - loss: 2.7350 - accuracy: 0.9788 - auc: 0.9976 - val_loss: 3.8827 - val_accuracy: 0.8806 - val_auc: 0.8194

Epoch 00025: val_auc did not improve from 0.84694
Epoch 26/100
271/271 - 12s - loss: 2.7305 - accuracy: 0.9809 - auc: 0.9976 - val_loss: 4.0497 - val_accuracy: 0.8922 - val_auc: 0.8200

Epoch 00026: val_auc did not improve from 0.84694
Epoch 27/100
271/271 - 12s - loss: 2.7277 - accuracy: 0.9806 - auc: 0.9977 - val_loss: 3.9599 - val_accuracy: 0.8839 - val_auc: 0.8087

Epoch 00027: val_auc did not improve from 0.84694
Epoch 28/100
271/271 - 13s - loss: 2.7210 - accuracy: 0.9832 - auc: 0.9981 - val_loss: 4.0121 - val_accuracy: 0.8839 - val_auc: 0.8163

Epoch 00028: val_auc did not improve from 0.84694
Epoch 29/100
271/27

271/271 - 12s - loss: 2.6249 - accuracy: 0.9935 - auc: 0.9991 - val_loss: 4.7310 - val_accuracy: 0.8988 - val_auc: 0.7777

Epoch 00068: val_auc did not improve from 0.84694
Epoch 69/100
271/271 - 12s - loss: 2.6236 - accuracy: 0.9932 - auc: 0.9994 - val_loss: 4.5554 - val_accuracy: 0.8939 - val_auc: 0.7943

Epoch 00069: val_auc did not improve from 0.84694
Epoch 70/100
271/271 - 12s - loss: 2.6200 - accuracy: 0.9947 - auc: 0.9991 - val_loss: 4.5934 - val_accuracy: 0.8955 - val_auc: 0.7772

Epoch 00070: val_auc did not improve from 0.84694
Epoch 71/100
271/271 - 12s - loss: 2.6183 - accuracy: 0.9947 - auc: 0.9991 - val_loss: 4.8203 - val_accuracy: 0.9038 - val_auc: 0.7602

Epoch 00071: val_auc did not improve from 0.84694
Epoch 72/100
271/271 - 12s - loss: 2.6168 - accuracy: 0.9942 - auc: 0.9994 - val_loss: 4.8153 - val_accuracy: 0.9055 - val_auc: 0.7881

Epoch 00072: val_auc did not improve from 0.84694
Epoch 73/100
271/271 - 12s - loss: 2.6166 - accuracy: 0.9941 - auc: 0.9990 - val_lo

In [53]:
attentions=[LS_model_.layers[1].attention_layers[i](train_tensor[:2]) 
            for i in range(n_attention)]


In [57]:
from tensorflow.keras.layers import Concatenate
concat_layer=Concatenate()
concat_layer(attentions)

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[6.44123197e-01, 1.00000000e+00, 1.00000000e+00, 2.25430576e-05,
        8.67243727e-07, 2.02709962e-05, 6.97454019e-03, 2.99754349e-04,
        3.12414329e-07, 2.31972504e-02, 4.19366241e-01, 7.46119804e-06,
        1.45811441e-06, 5.55445695e-05, 1.31393713e-03, 2.03859597e-03,
        4.51146298e-06, 9.99993205e-01, 1.00000000e+00, 2.88781479e-07],
       [6.25995040e-01, 1.00000000e+00, 1.00000000e+00, 5.70607190e-05,
        2.25496115e-06, 5.72950230e-05, 1.04652429e-02, 5.36933891e-04,
        1.15586545e-06, 2.92520169e-02, 4.23738629e-01, 8.87763417e-06,
        4.43622775e-06, 1.47782514e-04, 2.31930753e-03, 3.53392260e-03,
        1.23563905e-05, 9.99974847e-01, 9.99999762e-01, 9.02504041e-07]],
      dtype=float32)>

In [62]:
inputs=train_tensor[:2]
x=tf.einsum('ai, ak -> aik', concat_layer(attentions), inputs)
x=tf.reshape(x, (2,18900))
x

<tf.Tensor: shape=(2, 18900), dtype=float32, numpy=
array([[2.34226629e-01, 1.17113315e-01, 1.28824636e-01, ...,
        0.00000000e+00, 0.00000000e+00, 4.50371296e-08],
       [1.70725927e-01, 1.70725927e-01, 1.25199005e-01, ...,
        0.00000000e+00, 0.00000000e+00, 2.50676777e-08]], dtype=float32)>

In [ ]:
out=tf.math.dd(tf.tensordot(x, ))

In [65]:
np.min(LS_model_.layers[1].w1)

-0.034681212

In [70]:
init=tf.keras.initializers.GlorotNormal()
out=init(shape=(18900, 2048))

In [71]:
np.max(out)

0.022216508

In [72]:

np.min(out)

-0.022216508